# Install package

In [1]:
pip install 'gql[all]'

Note: you may need to restart the kernel to use updated packages.


In [13]:
import pandas as pd
import numpy as np
import datetime
from gql import gql, Client
from gql.transport.requests import RequestsHTTPTransport

# USDC/ETH 0.05% Pool

Fee Tier: 0.05%;
Tick Spacing: 10;
Token0: USD Coin;
Token1: Wrapped Ether.

In [2]:
sample_transport = RequestsHTTPTransport(
    url='https://api.thegraph.com/subgraphs/name/uniswap/uniswap-v3',
    verify=True,
    retries=5,
)
client = Client(
    transport=sample_transport
)
pool = "0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640"

## Mints

In [34]:
timestamp_lt = 1659283200 # Mon Aug 01 2022 00:00:00 GMT+0800 (中国标准时间)
timestamp_gt = 1627747200 # Sun Aug 01 2021 00:00:00 GMT+0800 (中国标准时间)
mint_list_data = []

while timestamp_lt > timestamp_gt:
    for i in range(0,6):
        skip = i * 1000
        query_body = '''
        {
            mints(skip:%i,first:1000,orderBy: timestamp, orderDirection: desc, where:
             { pool:"0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640",timestamp_lt:%i}
            ){
            transaction{
            blockNumber
            timestamp
            gasUsed
            gasPrice
            }
            amount0
            amount1
            amountUSD
            tickLower
            tickUpper
            }
            }
        '''%(skip,timestamp_lt)
        query_request = gql(query_body)
        if skip == 0:
            mint_origin_data = client.execute(query_request)
            mint_origin_data = mint_origin_data['mints']
        else:
            mint_origin_data.extend(client.execute(query_request)['mints'])
    mint_list_data.extend(mint_origin_data)
    timestamp_lt = int(mint_list_data[-1]['transaction']['timestamp'])

while int(mint_list_data[-1]['transaction']['timestamp']) < timestamp_gt:
    del mint_list_data[-1]
print(mint_list_data[-1]['transaction']['timestamp'])

1627753858


In [35]:
len(mint_list_data)

23159

Data Cleaning

In [36]:
df_mint = pd.DataFrame(mint_list_data)
df_mint.drop(df_mint.columns[0],axis=1,inplace=True)
list_blockNumber = []
list_timestamp = []
list_gasUsed = []
list_gasPrice = []
for i in range(0,len(mint_list_data)):
    list_blockNumber.append(int(mint_list_data[i]['transaction']['blockNumber']))
    list_timestamp.append(int(mint_list_data[i]['transaction']['timestamp']))
    list_gasUsed.append(int(mint_list_data[i]['transaction']['gasUsed']))
    list_gasPrice.append(int(mint_list_data[i]['transaction']['gasPrice']))
df_mint.insert(0,'timestamp',list_timestamp)
df_mint.insert(1,'blockNumber',list_blockNumber)
df_mint.insert(2,'gasUsed',list_gasUsed)
df_mint.insert(3,'gasPrice',list_gasPrice)
for i in range(0,len(mint_list_data)):
    df_mint['timestamp'][i] = datetime.datetime.fromtimestamp(df_mint['timestamp'][i])
df_mint.rename(columns={'timestamp':'Date'},inplace=True)
df_mint

C:\Users\Administrator\AppData\Local\Temp\ipykernel_16164\1135420938.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_mint['timestamp'][i] = datetime.datetime.fromtimestamp(df_mint['timestamp'][i])


,Date,blockNumber,gasUsed,gasPrice,amount0,amount1,amountUSD,tickLower,tickUpper
0,2022-07-31 23:56:31,15251187,579121,7838818730,2712.329386,1.574835586253042712,5404.06748324449068989288395235133,200790,202940
1,2022-07-31 23:20:12,15251010,569269,6024917966,819.566869,0.500209240618046763,1672.511478485068261416407917269959,201310,202480
2,2022-07-31 23:16:29,15250986,548653,9923781484,70,0.018583055074052271,101.6939833384548016110289290684265,201360,203190
3,2022-07-31 22:23:01,15250750,618345,9315834574,2405.918384,1.176946560374804643,4417.565866847275273418469073994895,201050,202890
4,2022-07-31 22:21:01,15250742,800000,6446767534,0,22804.19707494813972275,38977135.57152825314564010918934205,201870,201880
...,...,...,...,...,...,...,...,...,...
23154,2021-08-01 02:51:31,12935029,539518,84700000000,1359616.712763,0,1359616.712763,198250,198290
23155,2021-08-01 02:31:32,12934941,485630,65000000000,23633.452541,0,23633.452541,198270,198290
23156,2021-08-01 02:30:12,12934935,326499,70000000000,90477.45893,24.136582820517126153,149552.8700128114644873954661706415,196260,201430
23157,2021-08-01 02:27:13,12934922,564835,64000000000,9229.501964,3.496817041740419383,17788.12460575182371704055514156641,198080,198480


In [37]:
df_mint.columns

Index(['Date', 'blockNumber', 'gasUsed', 'gasPrice', 'amount0', 'amount1',
       'amountUSD', 'tickLower', 'tickUpper'],
      dtype='object')

In [16]:
df_mint.to_csv('tx_usdc_eth_500_mints.csv', index = False, header = True)

## Burns

In [38]:
timestamp_lt = 1659283200 # Mon Aug 01 2022 00:00:00 GMT+0800 (中国标准时间)
timestamp_gt = 1627747200 # Sun Aug 01 2021 00:00:00 GMT+0800 (中国标准时间)
burn_list_data = []

while timestamp_lt > timestamp_gt:
    for i in range(0,6):
        skip = i * 1000
        query_body = '''
        {
            burns(skip:%i,first:1000,orderBy: timestamp, orderDirection: desc, where:
             { pool:"0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640",timestamp_lt:%i}
            ){
            transaction{
            blockNumber
            timestamp
            gasUsed
            gasPrice
            }
            amount0
            amount1
            amountUSD
            tickLower
            tickUpper
            }
            }
        '''%(skip,timestamp_lt)
        query_request = gql(query_body)
        if skip == 0:
            burn_origin_data = client.execute(query_request)
            burn_origin_data = burn_origin_data['burns']
        else:
            burn_origin_data.extend(client.execute(query_request)['burns'])
    burn_list_data.extend(burn_origin_data)
    timestamp_lt = int(burn_list_data[-1]['transaction']['timestamp'])

while int(burn_list_data[-1]['transaction']['timestamp']) < timestamp_gt:
    del burn_list_data[-1]

print(burn_list_data[-1]['transaction']['timestamp'])
print(len(burn_list_data))

1627748013
25296


In [40]:
df_burn = pd.DataFrame(burn_list_data)
df_burn.drop(df_burn.columns[0],axis=1,inplace=True)
list_blockNumber = []
list_timestamp = []
list_gasUsed = []
list_gasPrice = []
for i in range(0,len(burn_list_data)):
    list_blockNumber.append(int(burn_list_data[i]['transaction']['blockNumber']))
    list_timestamp.append(int(burn_list_data[i]['transaction']['timestamp']))
    list_gasUsed.append(int(burn_list_data[i]['transaction']['gasUsed']))
    list_gasPrice.append(int(burn_list_data[i]['transaction']['gasPrice']))
df_burn.insert(0,'timestamp',list_timestamp)
df_burn.insert(1,'blockNumber',list_blockNumber)
df_burn.insert(2,'gasUsed',list_gasUsed)
df_burn.insert(3,'gasPrice',list_gasPrice)
for i in range(0,len(burn_list_data)):
    df_burn['timestamp'][i] = datetime.datetime.fromtimestamp(df_burn['timestamp'][i])
df_burn.rename(columns={'timestamp':'Date'},inplace=True)
df_burn

C:\Users\Administrator\AppData\Local\Temp\ipykernel_16164\3478827239.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_burn['timestamp'][i] = datetime.datetime.fromtimestamp(df_burn['timestamp'][i])


,Date,blockNumber,gasUsed,gasPrice,amount0,amount1,amountUSD,tickLower,tickUpper
0,2022-07-31 23:59:30,15251200,318334,7863801108,0,0,0,201240,204200
1,2022-07-31 23:47:39,15251143,289032,11162435326,4705.923172,0.257532373630133931,5145.709884293237322731660703760073,201640,204530
2,2022-07-31 23:09:17,15250952,302079,10670488317,311.033206,0.112854724817098693,503.51047918062871735512112803923,200060,204930
3,2022-07-31 22:46:46,15250849,297800,14367193632,0,0,0,200820,207240
4,2022-07-31 22:33:33,15250798,408042,9503481484,15141.504969,6.935377320774968871,27037.43627424270580092534079323728,201320,202500
...,...,...,...,...,...,...,...,...,...
25291,2021-08-01 03:06:21,12935084,297824,66000000000,0,0,0,198380,201010
25292,2021-08-01 02:50:05,12935024,480942,80300000000,1357686.309116,0,1357686.309116,198290,198580
25293,2021-08-01 02:23:27,12934904,395088,64000000000,32829.233123,3.050371619292884471,40291.84282902865931349260292653314,198080,199130
25294,2021-08-01 00:41:46,12934453,395241,42000000000,1170.45001,0.308788217230467147,1929.398764012756926248065327978175,197110,200060


In [17]:
df_burn.to_csv('tx_usdc_eth_500_burns.csv', index = False, header = True)

## Swaps

In [3]:
# From Sun Aug 01 2021 00:00:00 GMT+0800 To Mon Aug 01 2022 00:00:00 GMT+0800
time_list = [1659283200,1656604800,1655308800,1654012800,1652673600,1651334400,1648742400,1647403200,1646064000,1643644800,1640966400,1638288000,1635696000,1633017600,1630425600,1627747200]

In [55]:
(1656604800-1654012800)/2 + 1654012800

1655308800.0

In [5]:
(1654012800-1651334400)/2 + 1651334400

1652673600.0

In [6]:
(1648742400-1646064000) /2 + 1646064000

1647403200.0

In [8]:
len(time_list)

16

In [48]:
# swap_list_data = []

# for i in range(0,len(time_list)-2):
#     timestamp_lt = time_list[i]
#     timestamp_gt = time_list[i+1]
#     list_data = []
    
#     while timestamp_lt > timestamp_gt:
#         for i in range(0,6):
#             skip = i * 1000
#             query_body = '''
#             {
#                 swaps(skip:%i,first:1000,orderBy: timestamp, orderDirection: desc, where:
#                  { pool:"0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640",timestamp_lt:%i}
#                 ){
#                 transaction{
#                 blockNumber
#                 timestamp
#                 gasUsed
#                 gasPrice
#                 }
#                 amount0
#                 amount1
#                 amountUSD
#                 sqrtPriceX96
#                 tick
#                 }
#                 }
#             '''%(skip,timestamp_lt)
#             query_request = gql(query_body)
#             if skip == 0:
#                 swap_origin_data = client.execute(query_request)
#                 swap_origin_data = swap_origin_data['swaps']
#             else:
#                 swap_origin_data.extend(client.execute(query_request)['swaps'])
#         list_data.extend(swap_origin_data)
#         timestamp_lt = int(list_data[-1]['transaction']['timestamp'])

#     while int(list_data[-1]['transaction']['timestamp']) < timestamp_gt:
#         del list_data[-1]
    
#     swap_list_data.extend(list_data)
    
# print(swap_list_data[-1]['transaction']['timestamp'])
# print(len(swap_list_data))

TransportQueryError: {'message': 'Failed to get entities from store: canceling statement due to conflict with recovery, query = /* qid: 39436fb35ac130ff-dae7bacc875b284d */\nselect \'Transaction\' as entity, to_jsonb(c.*) as data from (\nselect c.*, p.id::text as g$parent_id\n/* child_type_d */ from rows from (unnest($1), unnest($2::text[])) as p(id, child_id), "sgd217942"."transaction" c where c.block_range @> $3 and c.id = p.child_id limit 1001) c\n  -- binds: [["0x00605b302afc138ebc040db288e867444f0a85b217ff821c6eb9193b90048b51#2508333", "0x015fe65c9f671c0b8f7d81bcea64f7272481a97ad679ee3d68cf053b0df6e6a3#2508596", "0x0171706658b51b5adf29a84d7ca4dc5ac6491d406d646a30b19c14b284687301#2509248", "0x01a54d49141250e578bd875b74928d76d5072c303712079017d63d88d5041ad1#2508544", "0x01db33ddfdbede8adbb84ea671f2a2c143b25660d00e1fa2ec26d3918aa5d11c#2508744", "0x0219004f5d2ddbead7fb6d40588efb783f2a008c7c7e51d3ddcc38adede5df7e#2508999", "0x023cbb714dcef27e0949741871f386df9ba50d5a88ec5e3680a3501ecf47

In [6]:
timestamp_lt = time_list[0]
timestamp_gt = time_list[1]
swap_list_data = []
while timestamp_lt > timestamp_gt:
    for i in range(0,6):
        skip = i * 1000
        query_body = '''
        {
            swaps(skip:%i,first:1000,orderBy: timestamp, orderDirection: desc, where:
             { pool:"0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640",timestamp_lt:%i}
            ){
            transaction{
            blockNumber
            timestamp
            gasUsed
            gasPrice
            }
            amount0
            amount1
            amountUSD
            sqrtPriceX96
            tick
            }
            }
        '''%(skip,timestamp_lt)
        query_request = gql(query_body)
        if skip == 0:
            swap_origin_data = client.execute(query_request)
            swap_origin_data = swap_origin_data['swaps']
        else:
            swap_origin_data.extend(client.execute(query_request)['swaps'])
    swap_list_data.extend(swap_origin_data)
    timestamp_lt = int(swap_list_data[-1]['transaction']['timestamp'])

while int(swap_list_data[-1]['transaction']['timestamp']) < timestamp_gt:
    del swap_list_data[-1]


print(swap_list_data[-1]['transaction']['timestamp'])
print(len(swap_list_data))

df_swap = pd.DataFrame(swap_list_data)
df_swap.drop(df_swap.columns[0],axis=1,inplace=True)
list_blockNumber = []
list_timestamp = []
list_gasUsed = []
list_gasPrice = []
for i in range(0,len(swap_list_data)):
    list_blockNumber.append(int(swap_list_data[i]['transaction']['blockNumber']))
    list_timestamp.append(int(swap_list_data[i]['transaction']['timestamp']))
    list_gasUsed.append(int(swap_list_data[i]['transaction']['gasUsed']))
    list_gasPrice.append(int(swap_list_data[i]['transaction']['gasPrice']))
df_swap.insert(0,'timestamp',list_timestamp)
df_swap.insert(1,'blockNumber',list_blockNumber)
df_swap.insert(2,'gasUsed',list_gasUsed)
df_swap.insert(3,'gasPrice',list_gasPrice)
for i in range(0,len(swap_list_data)):
    df_swap['timestamp'][i] = datetime.datetime.fromtimestamp(df_swap['timestamp'][i])
df_swap.rename(columns={'timestamp':'Date'},inplace=True)
df_swap.to_excel('tx_usdc_eth_500_swap01.xlsx', index = False, header = True)

1656604811
275222


C:\Users\Administrator\AppData\Local\Temp\ipykernel_10140\2622600028.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_swap['timestamp'][i] = datetime.datetime.fromtimestamp(df_swap['timestamp'][i])


In [7]:
timestamp_lt = time_list[1]
timestamp_gt = time_list[2]
swap_list_data = []
while timestamp_lt > timestamp_gt:
    for i in range(0,6):
        skip = i * 1000
        query_body = '''
        {
            swaps(skip:%i,first:1000,orderBy: timestamp, orderDirection: desc, where:
             { pool:"0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640",timestamp_lt:%i}
            ){
            transaction{
            blockNumber
            timestamp
            gasUsed
            gasPrice
            }
            amount0
            amount1
            amountUSD
            sqrtPriceX96
            tick
            }
            }
        '''%(skip,timestamp_lt)
        query_request = gql(query_body)
        if skip == 0:
            swap_origin_data = client.execute(query_request)
            swap_origin_data = swap_origin_data['swaps']
        else:
            swap_origin_data.extend(client.execute(query_request)['swaps'])
    swap_list_data.extend(swap_origin_data)
    timestamp_lt = int(swap_list_data[-1]['transaction']['timestamp'])

while int(swap_list_data[-1]['transaction']['timestamp']) < timestamp_gt:
    del swap_list_data[-1]

print(swap_list_data[-1]['transaction']['timestamp'])
print(len(swap_list_data))

df_swap = pd.DataFrame(swap_list_data)
df_swap.drop(df_swap.columns[0],axis=1,inplace=True)
list_blockNumber = []
list_timestamp = []
list_gasUsed = []
list_gasPrice = []
for i in range(0,len(swap_list_data)):
    list_blockNumber.append(int(swap_list_data[i]['transaction']['blockNumber']))
    list_timestamp.append(int(swap_list_data[i]['transaction']['timestamp']))
    list_gasUsed.append(int(swap_list_data[i]['transaction']['gasUsed']))
    list_gasPrice.append(int(swap_list_data[i]['transaction']['gasPrice']))
df_swap.insert(0,'timestamp',list_timestamp)
df_swap.insert(1,'blockNumber',list_blockNumber)
df_swap.insert(2,'gasUsed',list_gasUsed)
df_swap.insert(3,'gasPrice',list_gasPrice)
for i in range(0,len(swap_list_data)):
    df_swap['timestamp'][i] = datetime.datetime.fromtimestamp(df_swap['timestamp'][i])
df_swap.rename(columns={'timestamp':'Date'},inplace=True)
df_swap.to_excel('tx_usdc_eth_500_swap02.xlsx', index = False, header = True)

1655308820
125359


C:\Users\Administrator\AppData\Local\Temp\ipykernel_10140\4267658656.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_swap['timestamp'][i] = datetime.datetime.fromtimestamp(df_swap['timestamp'][i])


In [8]:
timestamp_lt = time_list[2]
timestamp_gt = time_list[3]
swap_list_data = []
while timestamp_lt > timestamp_gt:
    for i in range(0,6):
        skip = i * 1000
        query_body = '''
        {
            swaps(skip:%i,first:1000,orderBy: timestamp, orderDirection: desc, where:
             { pool:"0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640",timestamp_lt:%i}
            ){
            transaction{
            blockNumber
            timestamp
            gasUsed
            gasPrice
            }
            amount0
            amount1
            amountUSD
            sqrtPriceX96
            tick
            }
            }
        '''%(skip,timestamp_lt)
        query_request = gql(query_body)
        if skip == 0:
            swap_origin_data = client.execute(query_request)
            swap_origin_data = swap_origin_data['swaps']
        else:
            swap_origin_data.extend(client.execute(query_request)['swaps'])
    swap_list_data.extend(swap_origin_data)
    timestamp_lt = int(swap_list_data[-1]['transaction']['timestamp'])

while int(swap_list_data[-1]['transaction']['timestamp']) < timestamp_gt:
    del swap_list_data[-1]

print(swap_list_data[-1]['transaction']['timestamp'])
print(len(swap_list_data))

df_swap = pd.DataFrame(swap_list_data)
df_swap.drop(df_swap.columns[0],axis=1,inplace=True)
list_blockNumber = []
list_timestamp = []
list_gasUsed = []
list_gasPrice = []
for i in range(0,len(swap_list_data)):
    list_blockNumber.append(int(swap_list_data[i]['transaction']['blockNumber']))
    list_timestamp.append(int(swap_list_data[i]['transaction']['timestamp']))
    list_gasUsed.append(int(swap_list_data[i]['transaction']['gasUsed']))
    list_gasPrice.append(int(swap_list_data[i]['transaction']['gasPrice']))
df_swap.insert(0,'timestamp',list_timestamp)
df_swap.insert(1,'blockNumber',list_blockNumber)
df_swap.insert(2,'gasUsed',list_gasUsed)
df_swap.insert(3,'gasPrice',list_gasPrice)
for i in range(0,len(swap_list_data)):
    df_swap['timestamp'][i] = datetime.datetime.fromtimestamp(df_swap['timestamp'][i])
df_swap.rename(columns={'timestamp':'Date'},inplace=True)
df_swap.to_excel('tx_usdc_eth_500_swap03.xlsx', index = False, header = True)

1654012849
116320


C:\Users\Administrator\AppData\Local\Temp\ipykernel_10140\791530257.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_swap['timestamp'][i] = datetime.datetime.fromtimestamp(df_swap['timestamp'][i])


In [8]:
timestamp_lt = time_list[3]
timestamp_gt = time_list[4]
swap_list_data = []
while timestamp_lt > timestamp_gt:
    for i in range(0,6):
        skip = i * 1000
        query_body = '''
        {
            swaps(skip:%i,first:1000,orderBy: timestamp, orderDirection: desc, where:
             { pool:"0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640",timestamp_lt:%i}
            ){
            transaction{
            blockNumber
            timestamp
            gasUsed
            gasPrice
            }
            amount0
            amount1
            amountUSD
            sqrtPriceX96
            tick
            }
            }
        '''%(skip,timestamp_lt)
        query_request = gql(query_body)
        if skip == 0:
            swap_origin_data = client.execute(query_request)
            swap_origin_data = swap_origin_data['swaps']
        else:
            swap_origin_data.extend(client.execute(query_request)['swaps'])
    swap_list_data.extend(swap_origin_data)
    timestamp_lt = int(swap_list_data[-1]['transaction']['timestamp'])

while int(swap_list_data[-1]['transaction']['timestamp']) < timestamp_gt:
    del swap_list_data[-1]

print(swap_list_data[-1]['transaction']['timestamp'])
print(len(swap_list_data))

df_swap = pd.DataFrame(swap_list_data)
df_swap.drop(df_swap.columns[0],axis=1,inplace=True)
list_blockNumber = []
list_timestamp = []
list_gasUsed = []
list_gasPrice = []
for i in range(0,len(swap_list_data)):
    list_blockNumber.append(int(swap_list_data[i]['transaction']['blockNumber']))
    list_timestamp.append(int(swap_list_data[i]['transaction']['timestamp']))
    list_gasUsed.append(int(swap_list_data[i]['transaction']['gasUsed']))
    list_gasPrice.append(int(swap_list_data[i]['transaction']['gasPrice']))
df_swap.insert(0,'timestamp',list_timestamp)
df_swap.insert(1,'blockNumber',list_blockNumber)
df_swap.insert(2,'gasUsed',list_gasUsed)
df_swap.insert(3,'gasPrice',list_gasPrice)
for i in range(0,len(swap_list_data)):
    df_swap['timestamp'][i] = datetime.datetime.fromtimestamp(df_swap['timestamp'][i])
df_swap.rename(columns={'timestamp':'Date'},inplace=True)
df_swap.to_excel('tx_usdc_eth_500_swap04.xlsx', index = False, header = True)

1652673636
101854


C:\Users\Administrator\AppData\Local\Temp\ipykernel_18456\379641520.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_swap['timestamp'][i] = datetime.datetime.fromtimestamp(df_swap['timestamp'][i])


In [9]:
timestamp_lt = time_list[4]
timestamp_gt = time_list[5]
swap_list_data = []
while timestamp_lt > timestamp_gt:
    for i in range(0,6):
        skip = i * 1000
        query_body = '''
        {
            swaps(skip:%i,first:1000,orderBy: timestamp, orderDirection: desc, where:
             { pool:"0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640",timestamp_lt:%i}
            ){
            transaction{
            blockNumber
            timestamp
            gasUsed
            gasPrice
            }
            amount0
            amount1
            amountUSD
            sqrtPriceX96
            tick
            }
            }
        '''%(skip,timestamp_lt)
        query_request = gql(query_body)
        if skip == 0:
            swap_origin_data = client.execute(query_request)
            swap_origin_data = swap_origin_data['swaps']
        else:
            swap_origin_data.extend(client.execute(query_request)['swaps'])
    swap_list_data.extend(swap_origin_data)
    timestamp_lt = int(swap_list_data[-1]['transaction']['timestamp'])

while int(swap_list_data[-1]['transaction']['timestamp']) < timestamp_gt:
    del swap_list_data[-1]

print(swap_list_data[-1]['transaction']['timestamp'])
print(len(swap_list_data))

df_swap = pd.DataFrame(swap_list_data)
df_swap.drop(df_swap.columns[0],axis=1,inplace=True)
list_blockNumber = []
list_timestamp = []
list_gasUsed = []
list_gasPrice = []
for i in range(0,len(swap_list_data)):
    list_blockNumber.append(int(swap_list_data[i]['transaction']['blockNumber']))
    list_timestamp.append(int(swap_list_data[i]['transaction']['timestamp']))
    list_gasUsed.append(int(swap_list_data[i]['transaction']['gasUsed']))
    list_gasPrice.append(int(swap_list_data[i]['transaction']['gasPrice']))
df_swap.insert(0,'timestamp',list_timestamp)
df_swap.insert(1,'blockNumber',list_blockNumber)
df_swap.insert(2,'gasUsed',list_gasUsed)
df_swap.insert(3,'gasPrice',list_gasPrice)
for i in range(0,len(swap_list_data)):
    df_swap['timestamp'][i] = datetime.datetime.fromtimestamp(df_swap['timestamp'][i])
df_swap.rename(columns={'timestamp':'Date'},inplace=True)
df_swap.to_excel('tx_usdc_eth_500_swap05.xlsx', index = False, header = True)

1651334403
124411


C:\Users\Administrator\AppData\Local\Temp\ipykernel_18456\1362249426.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_swap['timestamp'][i] = datetime.datetime.fromtimestamp(df_swap['timestamp'][i])


In [4]:
timestamp_lt = time_list[5]
timestamp_gt = time_list[6]
swap_list_data = []
while timestamp_lt > timestamp_gt:
    for i in range(0,6):
        skip = i * 1000
        query_body = '''
        {
            swaps(skip:%i,first:1000,orderBy: timestamp, orderDirection: desc, where:
             { pool:"0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640",timestamp_lt:%i}
            ){
            transaction{
            blockNumber
            timestamp
            gasUsed
            gasPrice
            }
            amount0
            amount1
            amountUSD
            sqrtPriceX96
            tick
            }
            }
        '''%(skip,timestamp_lt)
        query_request = gql(query_body)
        if skip == 0:
            swap_origin_data = client.execute(query_request)
            swap_origin_data = swap_origin_data['swaps']
        else:
            swap_origin_data.extend(client.execute(query_request)['swaps'])
    swap_list_data.extend(swap_origin_data)
    timestamp_lt = int(swap_list_data[-1]['transaction']['timestamp'])

while int(swap_list_data[-1]['transaction']['timestamp']) < timestamp_gt:
    del swap_list_data[-1]

print(swap_list_data[-1]['transaction']['timestamp'])
print(len(swap_list_data))

df_swap = pd.DataFrame(swap_list_data)
df_swap.drop(df_swap.columns[0],axis=1,inplace=True)
list_blockNumber = []
list_timestamp = []
list_gasUsed = []
list_gasPrice = []
for i in range(0,len(swap_list_data)):
    list_blockNumber.append(int(swap_list_data[i]['transaction']['blockNumber']))
    list_timestamp.append(int(swap_list_data[i]['transaction']['timestamp']))
    list_gasUsed.append(int(swap_list_data[i]['transaction']['gasUsed']))
    list_gasPrice.append(int(swap_list_data[i]['transaction']['gasPrice']))
df_swap.insert(0,'timestamp',list_timestamp)
df_swap.insert(1,'blockNumber',list_blockNumber)
df_swap.insert(2,'gasUsed',list_gasUsed)
df_swap.insert(3,'gasPrice',list_gasPrice)
for i in range(0,len(swap_list_data)):
    df_swap['timestamp'][i] = datetime.datetime.fromtimestamp(df_swap['timestamp'][i])
df_swap.rename(columns={'timestamp':'Date'},inplace=True)
df_swap.to_excel('tx_usdc_eth_500_swap06.xlsx', index = False, header = True)

1648742408
156404


C:\Users\Administrator\AppData\Local\Temp\ipykernel_10780\1991966714.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_swap['timestamp'][i] = datetime.datetime.fromtimestamp(df_swap['timestamp'][i])


In [9]:
timestamp_lt = time_list[6]
timestamp_gt = time_list[7]
swap_list_data = []
while timestamp_lt > timestamp_gt:
    for i in range(0,6):
        skip = i * 1000
        query_body = '''
        {
            swaps(skip:%i,first:1000,orderBy: timestamp, orderDirection: desc, where:
             { pool:"0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640",timestamp_lt:%i}
            ){
            transaction{
            blockNumber
            timestamp
            gasUsed
            gasPrice
            }
            amount0
            amount1
            amountUSD
            sqrtPriceX96
            tick
            }
            }
        '''%(skip,timestamp_lt)
        query_request = gql(query_body)
        if skip == 0:
            swap_origin_data = client.execute(query_request)
            swap_origin_data = swap_origin_data['swaps']
        else:
            swap_origin_data.extend(client.execute(query_request)['swaps'])
    swap_list_data.extend(swap_origin_data)
    timestamp_lt = int(swap_list_data[-1]['transaction']['timestamp'])

while int(swap_list_data[-1]['transaction']['timestamp']) < timestamp_gt:
    del swap_list_data[-1]

print(swap_list_data[-1]['transaction']['timestamp'])
print(len(swap_list_data))

df_swap = pd.DataFrame(swap_list_data)
df_swap.drop(df_swap.columns[0],axis=1,inplace=True)
list_blockNumber = []
list_timestamp = []
list_gasUsed = []
list_gasPrice = []
for i in range(0,len(swap_list_data)):
    list_blockNumber.append(int(swap_list_data[i]['transaction']['blockNumber']))
    list_timestamp.append(int(swap_list_data[i]['transaction']['timestamp']))
    list_gasUsed.append(int(swap_list_data[i]['transaction']['gasUsed']))
    list_gasPrice.append(int(swap_list_data[i]['transaction']['gasPrice']))
df_swap.insert(0,'timestamp',list_timestamp)
df_swap.insert(1,'blockNumber',list_blockNumber)
df_swap.insert(2,'gasUsed',list_gasUsed)
df_swap.insert(3,'gasPrice',list_gasPrice)
for i in range(0,len(swap_list_data)):
    df_swap['timestamp'][i] = datetime.datetime.fromtimestamp(df_swap['timestamp'][i])
df_swap.rename(columns={'timestamp':'Date'},inplace=True)
df_swap.to_excel('tx_usdc_eth_500_swap07.xlsx', index = False, header = True)

1647403204
91291


C:\Users\Administrator\AppData\Local\Temp\ipykernel_10044\2354835462.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_swap['timestamp'][i] = datetime.datetime.fromtimestamp(df_swap['timestamp'][i])


In [10]:
timestamp_lt = time_list[7]
timestamp_gt = time_list[8]
swap_list_data = []
while timestamp_lt > timestamp_gt:
    for i in range(0,6):
        skip = i * 1000
        query_body = '''
        {
            swaps(skip:%i,first:1000,orderBy: timestamp, orderDirection: desc, where:
             { pool:"0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640",timestamp_lt:%i}
            ){
            transaction{
            blockNumber
            timestamp
            gasUsed
            gasPrice
            }
            amount0
            amount1
            amountUSD
            sqrtPriceX96
            tick
            }
            }
        '''%(skip,timestamp_lt)
        query_request = gql(query_body)
        if skip == 0:
            swap_origin_data = client.execute(query_request)
            swap_origin_data = swap_origin_data['swaps']
        else:
            swap_origin_data.extend(client.execute(query_request)['swaps'])
    swap_list_data.extend(swap_origin_data)
    timestamp_lt = int(swap_list_data[-1]['transaction']['timestamp'])

while int(swap_list_data[-1]['transaction']['timestamp']) < timestamp_gt:
    del swap_list_data[-1]

print(swap_list_data[-1]['transaction']['timestamp'])
print(len(swap_list_data))

df_swap = pd.DataFrame(swap_list_data)
df_swap.drop(df_swap.columns[0],axis=1,inplace=True)
list_blockNumber = []
list_timestamp = []
list_gasUsed = []
list_gasPrice = []
for i in range(0,len(swap_list_data)):
    list_blockNumber.append(int(swap_list_data[i]['transaction']['blockNumber']))
    list_timestamp.append(int(swap_list_data[i]['transaction']['timestamp']))
    list_gasUsed.append(int(swap_list_data[i]['transaction']['gasUsed']))
    list_gasPrice.append(int(swap_list_data[i]['transaction']['gasPrice']))
df_swap.insert(0,'timestamp',list_timestamp)
df_swap.insert(1,'blockNumber',list_blockNumber)
df_swap.insert(2,'gasUsed',list_gasUsed)
df_swap.insert(3,'gasPrice',list_gasPrice)
for i in range(0,len(swap_list_data)):
    df_swap['timestamp'][i] = datetime.datetime.fromtimestamp(df_swap['timestamp'][i])
df_swap.rename(columns={'timestamp':'Date'},inplace=True)
df_swap.to_excel('tx_usdc_eth_500_swap08.xlsx', index = False, header = True)

1646064011
93918


C:\Users\Administrator\AppData\Local\Temp\ipykernel_10044\114951888.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_swap['timestamp'][i] = datetime.datetime.fromtimestamp(df_swap['timestamp'][i])


In [4]:
timestamp_lt = time_list[8]
timestamp_gt = time_list[9]
swap_list_data = []
while timestamp_lt > timestamp_gt:
    for i in range(0,6):
        skip = i * 1000
        query_body = '''
        {
            swaps(skip:%i,first:1000,orderBy: timestamp, orderDirection: desc, where:
             { pool:"0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640",timestamp_lt:%i}
            ){
            transaction{
            blockNumber
            timestamp
            gasUsed
            gasPrice
            }
            amount0
            amount1
            amountUSD
            sqrtPriceX96
            tick
            }
            }
        '''%(skip,timestamp_lt)
        query_request = gql(query_body)
        if skip == 0:
            swap_origin_data = client.execute(query_request)
            swap_origin_data = swap_origin_data['swaps']
        else:
            swap_origin_data.extend(client.execute(query_request)['swaps'])
    swap_list_data.extend(swap_origin_data)
    timestamp_lt = int(swap_list_data[-1]['transaction']['timestamp'])

while int(swap_list_data[-1]['transaction']['timestamp']) < timestamp_gt:
    del swap_list_data[-1]

print(swap_list_data[-1]['transaction']['timestamp'])
print(len(swap_list_data))

df_swap = pd.DataFrame(swap_list_data)
df_swap.drop(df_swap.columns[0],axis=1,inplace=True)
list_blockNumber = []
list_timestamp = []
list_gasUsed = []
list_gasPrice = []
for i in range(0,len(swap_list_data)):
    list_blockNumber.append(int(swap_list_data[i]['transaction']['blockNumber']))
    list_timestamp.append(int(swap_list_data[i]['transaction']['timestamp']))
    list_gasUsed.append(int(swap_list_data[i]['transaction']['gasUsed']))
    list_gasPrice.append(int(swap_list_data[i]['transaction']['gasPrice']))
df_swap.insert(0,'timestamp',list_timestamp)
df_swap.insert(1,'blockNumber',list_blockNumber)
df_swap.insert(2,'gasUsed',list_gasUsed)
df_swap.insert(3,'gasPrice',list_gasPrice)
for i in range(0,len(swap_list_data)):
    df_swap['timestamp'][i] = datetime.datetime.fromtimestamp(df_swap['timestamp'][i])
df_swap.rename(columns={'timestamp':'Date'},inplace=True)
df_swap.to_excel('tx_usdc_eth_500_swap09.xlsx', index = False, header = True)

1643644803
146076


C:\Users\Administrator\AppData\Local\Temp\ipykernel_4404\3402370298.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_swap['timestamp'][i] = datetime.datetime.fromtimestamp(df_swap['timestamp'][i])


In [5]:
timestamp_lt = time_list[9]
timestamp_gt = time_list[10]
swap_list_data = []
while timestamp_lt > timestamp_gt:
    for i in range(0,6):
        skip = i * 1000
        query_body = '''
        {
            swaps(skip:%i,first:1000,orderBy: timestamp, orderDirection: desc, where:
             { pool:"0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640",timestamp_lt:%i}
            ){
            transaction{
            blockNumber
            timestamp
            gasUsed
            gasPrice
            }
            amount0
            amount1
            amountUSD
            sqrtPriceX96
            tick
            }
            }
        '''%(skip,timestamp_lt)
        query_request = gql(query_body)
        if skip == 0:
            swap_origin_data = client.execute(query_request)
            swap_origin_data = swap_origin_data['swaps']
        else:
            swap_origin_data.extend(client.execute(query_request)['swaps'])
    swap_list_data.extend(swap_origin_data)
    timestamp_lt = int(swap_list_data[-1]['transaction']['timestamp'])

while int(swap_list_data[-1]['transaction']['timestamp']) < timestamp_gt:
    del swap_list_data[-1]

print(swap_list_data[-1]['transaction']['timestamp'])
print(len(swap_list_data))

df_swap = pd.DataFrame(swap_list_data)
df_swap.drop(df_swap.columns[0],axis=1,inplace=True)
list_blockNumber = []
list_timestamp = []
list_gasUsed = []
list_gasPrice = []
for i in range(0,len(swap_list_data)):
    list_blockNumber.append(int(swap_list_data[i]['transaction']['blockNumber']))
    list_timestamp.append(int(swap_list_data[i]['transaction']['timestamp']))
    list_gasUsed.append(int(swap_list_data[i]['transaction']['gasUsed']))
    list_gasPrice.append(int(swap_list_data[i]['transaction']['gasPrice']))
df_swap.insert(0,'timestamp',list_timestamp)
df_swap.insert(1,'blockNumber',list_blockNumber)
df_swap.insert(2,'gasUsed',list_gasUsed)
df_swap.insert(3,'gasPrice',list_gasPrice)
for i in range(0,len(swap_list_data)):
    df_swap['timestamp'][i] = datetime.datetime.fromtimestamp(df_swap['timestamp'][i])
df_swap.rename(columns={'timestamp':'Date'},inplace=True)
df_swap.to_excel('tx_usdc_eth_500_swap10.xlsx', index = False, header = True)

1640966401
162087


C:\Users\Administrator\AppData\Local\Temp\ipykernel_4404\491852276.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_swap['timestamp'][i] = datetime.datetime.fromtimestamp(df_swap['timestamp'][i])


In [4]:
timestamp_lt = time_list[10]
timestamp_gt = time_list[11]
swap_list_data = []
while timestamp_lt > timestamp_gt:
    for i in range(0,6):
        skip = i * 1000
        query_body = '''
        {
            swaps(skip:%i,first:1000,orderBy: timestamp, orderDirection: desc, where:
             { pool:"0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640",timestamp_lt:%i}
            ){
            transaction{
            blockNumber
            timestamp
            gasUsed
            gasPrice
            }
            amount0
            amount1
            amountUSD
            sqrtPriceX96
            tick
            }
            }
        '''%(skip,timestamp_lt)
        query_request = gql(query_body)
        if skip == 0:
            swap_origin_data = client.execute(query_request)
            swap_origin_data = swap_origin_data['swaps']
        else:
            swap_origin_data.extend(client.execute(query_request)['swaps'])
    swap_list_data.extend(swap_origin_data)
    timestamp_lt = int(swap_list_data[-1]['transaction']['timestamp'])

while int(swap_list_data[-1]['transaction']['timestamp']) < timestamp_gt:
    del swap_list_data[-1]

print(swap_list_data[-1]['transaction']['timestamp'])
print(len(swap_list_data))

df_swap = pd.DataFrame(swap_list_data)
df_swap.drop(df_swap.columns[0],axis=1,inplace=True)
list_blockNumber = []
list_timestamp = []
list_gasUsed = []
list_gasPrice = []
for i in range(0,len(swap_list_data)):
    list_blockNumber.append(int(swap_list_data[i]['transaction']['blockNumber']))
    list_timestamp.append(int(swap_list_data[i]['transaction']['timestamp']))
    list_gasUsed.append(int(swap_list_data[i]['transaction']['gasUsed']))
    list_gasPrice.append(int(swap_list_data[i]['transaction']['gasPrice']))
df_swap.insert(0,'timestamp',list_timestamp)
df_swap.insert(1,'blockNumber',list_blockNumber)
df_swap.insert(2,'gasUsed',list_gasUsed)
df_swap.insert(3,'gasPrice',list_gasPrice)
for i in range(0,len(swap_list_data)):
    df_swap['timestamp'][i] = datetime.datetime.fromtimestamp(df_swap['timestamp'][i])
df_swap.rename(columns={'timestamp':'Date'},inplace=True)
df_swap.to_excel('tx_usdc_eth_500_swap11.xlsx', index = False, header = True)

1638288002
158154


C:\Users\Administrator\AppData\Local\Temp\ipykernel_23272\2850321077.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_swap['timestamp'][i] = datetime.datetime.fromtimestamp(df_swap['timestamp'][i])


In [5]:
timestamp_lt = time_list[11]
timestamp_gt = time_list[12]
swap_list_data = []
while timestamp_lt > timestamp_gt:
    for i in range(0,6):
        skip = i * 1000
        query_body = '''
        {
            swaps(skip:%i,first:1000,orderBy: timestamp, orderDirection: desc, where:
             { pool:"0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640",timestamp_lt:%i}
            ){
            transaction{
            blockNumber
            timestamp
            gasUsed
            gasPrice
            }
            amount0
            amount1
            amountUSD
            sqrtPriceX96
            tick
            }
            }
        '''%(skip,timestamp_lt)
        query_request = gql(query_body)
        if skip == 0:
            swap_origin_data = client.execute(query_request)
            swap_origin_data = swap_origin_data['swaps']
        else:
            swap_origin_data.extend(client.execute(query_request)['swaps'])
    swap_list_data.extend(swap_origin_data)
    timestamp_lt = int(swap_list_data[-1]['transaction']['timestamp'])

while int(swap_list_data[-1]['transaction']['timestamp']) < timestamp_gt:
    del swap_list_data[-1]

print(swap_list_data[-1]['transaction']['timestamp'])
print(len(swap_list_data))

df_swap = pd.DataFrame(swap_list_data)
df_swap.drop(df_swap.columns[0],axis=1,inplace=True)
list_blockNumber = []
list_timestamp = []
list_gasUsed = []
list_gasPrice = []
for i in range(0,len(swap_list_data)):
    list_blockNumber.append(int(swap_list_data[i]['transaction']['blockNumber']))
    list_timestamp.append(int(swap_list_data[i]['transaction']['timestamp']))
    list_gasUsed.append(int(swap_list_data[i]['transaction']['gasUsed']))
    list_gasPrice.append(int(swap_list_data[i]['transaction']['gasPrice']))
df_swap.insert(0,'timestamp',list_timestamp)
df_swap.insert(1,'blockNumber',list_blockNumber)
df_swap.insert(2,'gasUsed',list_gasUsed)
df_swap.insert(3,'gasPrice',list_gasPrice)
for i in range(0,len(swap_list_data)):
    df_swap['timestamp'][i] = datetime.datetime.fromtimestamp(df_swap['timestamp'][i])
df_swap.rename(columns={'timestamp':'Date'},inplace=True)
df_swap.to_excel('tx_usdc_eth_500_swap12.xlsx', index = False, header = True)

1635696139
125969


C:\Users\Administrator\AppData\Local\Temp\ipykernel_23272\1558202833.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_swap['timestamp'][i] = datetime.datetime.fromtimestamp(df_swap['timestamp'][i])


In [6]:
timestamp_lt = time_list[12]
timestamp_gt = time_list[13]
swap_list_data = []
while timestamp_lt > timestamp_gt:
    for i in range(0,6):
        skip = i * 1000
        query_body = '''
        {
            swaps(skip:%i,first:1000,orderBy: timestamp, orderDirection: desc, where:
             { pool:"0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640",timestamp_lt:%i}
            ){
            transaction{
            blockNumber
            timestamp
            gasUsed
            gasPrice
            }
            amount0
            amount1
            amountUSD
            sqrtPriceX96
            tick
            }
            }
        '''%(skip,timestamp_lt)
        query_request = gql(query_body)
        if skip == 0:
            swap_origin_data = client.execute(query_request)
            swap_origin_data = swap_origin_data['swaps']
        else:
            swap_origin_data.extend(client.execute(query_request)['swaps'])
    swap_list_data.extend(swap_origin_data)
    timestamp_lt = int(swap_list_data[-1]['transaction']['timestamp'])

while int(swap_list_data[-1]['transaction']['timestamp']) < timestamp_gt:
    del swap_list_data[-1]

print(swap_list_data[-1]['transaction']['timestamp'])
print(len(swap_list_data))

df_swap = pd.DataFrame(swap_list_data)
df_swap.drop(df_swap.columns[0],axis=1,inplace=True)
list_blockNumber = []
list_timestamp = []
list_gasUsed = []
list_gasPrice = []
for i in range(0,len(swap_list_data)):
    list_blockNumber.append(int(swap_list_data[i]['transaction']['blockNumber']))
    list_timestamp.append(int(swap_list_data[i]['transaction']['timestamp']))
    list_gasUsed.append(int(swap_list_data[i]['transaction']['gasUsed']))
    list_gasPrice.append(int(swap_list_data[i]['transaction']['gasPrice']))
df_swap.insert(0,'timestamp',list_timestamp)
df_swap.insert(1,'blockNumber',list_blockNumber)
df_swap.insert(2,'gasUsed',list_gasUsed)
df_swap.insert(3,'gasPrice',list_gasPrice)
for i in range(0,len(swap_list_data)):
    df_swap['timestamp'][i] = datetime.datetime.fromtimestamp(df_swap['timestamp'][i])
df_swap.rename(columns={'timestamp':'Date'},inplace=True)
df_swap.to_excel('tx_usdc_eth_500_swap13.xlsx', index = False, header = True)

1633017601
120473


C:\Users\Administrator\AppData\Local\Temp\ipykernel_23272\4023155723.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_swap['timestamp'][i] = datetime.datetime.fromtimestamp(df_swap['timestamp'][i])


In [4]:
timestamp_lt = time_list[13]
timestamp_gt = time_list[14]
swap_list_data = []
while timestamp_lt > timestamp_gt:
    for i in range(0,6):
        skip = i * 1000
        query_body = '''
        {
            swaps(skip:%i,first:1000,orderBy: timestamp, orderDirection: desc, where:
             { pool:"0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640",timestamp_lt:%i}
            ){
            transaction{
            blockNumber
            timestamp
            gasUsed
            gasPrice
            }
            amount0
            amount1
            amountUSD
            sqrtPriceX96
            tick
            }
            }
        '''%(skip,timestamp_lt)
        query_request = gql(query_body)
        if skip == 0:
            swap_origin_data = client.execute(query_request)
            swap_origin_data = swap_origin_data['swaps']
        else:
            swap_origin_data.extend(client.execute(query_request)['swaps'])
    swap_list_data.extend(swap_origin_data)
    timestamp_lt = int(swap_list_data[-1]['transaction']['timestamp'])

while int(swap_list_data[-1]['transaction']['timestamp']) < timestamp_gt:
    del swap_list_data[-1]

print(swap_list_data[-1]['transaction']['timestamp'])
print(len(swap_list_data))

df_swap = pd.DataFrame(swap_list_data)
df_swap.drop(df_swap.columns[0],axis=1,inplace=True)
list_blockNumber = []
list_timestamp = []
list_gasUsed = []
list_gasPrice = []
for i in range(0,len(swap_list_data)):
    list_blockNumber.append(int(swap_list_data[i]['transaction']['blockNumber']))
    list_timestamp.append(int(swap_list_data[i]['transaction']['timestamp']))
    list_gasUsed.append(int(swap_list_data[i]['transaction']['gasUsed']))
    list_gasPrice.append(int(swap_list_data[i]['transaction']['gasPrice']))
df_swap.insert(0,'timestamp',list_timestamp)
df_swap.insert(1,'blockNumber',list_blockNumber)
df_swap.insert(2,'gasUsed',list_gasUsed)
df_swap.insert(3,'gasPrice',list_gasPrice)
for i in range(0,len(swap_list_data)):
    df_swap['timestamp'][i] = datetime.datetime.fromtimestamp(df_swap['timestamp'][i])
df_swap.rename(columns={'timestamp':'Date'},inplace=True)
df_swap.to_excel('tx_usdc_eth_500_swap14.xlsx', index = False, header = True)

1630425630
137753


C:\Users\Administrator\AppData\Local\Temp\ipykernel_4404\3105732539.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_swap['timestamp'][i] = datetime.datetime.fromtimestamp(df_swap['timestamp'][i])


In [5]:
timestamp_lt = time_list[14]
timestamp_gt = time_list[15]
swap_list_data = []
while timestamp_lt > timestamp_gt:
    for i in range(0,6):
        skip = i * 1000
        query_body = '''
        {
            swaps(skip:%i,first:1000,orderBy: timestamp, orderDirection: desc, where:
             { pool:"0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640",timestamp_lt:%i}
            ){
            transaction{
            blockNumber
            timestamp
            gasUsed
            gasPrice
            }
            amount0
            amount1
            amountUSD
            sqrtPriceX96
            tick
            }
            }
        '''%(skip,timestamp_lt)
        query_request = gql(query_body)
        if skip == 0:
            swap_origin_data = client.execute(query_request)
            swap_origin_data = swap_origin_data['swaps']
        else:
            swap_origin_data.extend(client.execute(query_request)['swaps'])
    swap_list_data.extend(swap_origin_data)
    timestamp_lt = int(swap_list_data[-1]['transaction']['timestamp'])

while int(swap_list_data[-1]['transaction']['timestamp']) < timestamp_gt:
    del swap_list_data[-1]

print(swap_list_data[-1]['transaction']['timestamp'])
print(len(swap_list_data))

df_swap = pd.DataFrame(swap_list_data)
df_swap.drop(df_swap.columns[0],axis=1,inplace=True)
list_blockNumber = []
list_timestamp = []
list_gasUsed = []
list_gasPrice = []
for i in range(0,len(swap_list_data)):
    list_blockNumber.append(int(swap_list_data[i]['transaction']['blockNumber']))
    list_timestamp.append(int(swap_list_data[i]['transaction']['timestamp']))
    list_gasUsed.append(int(swap_list_data[i]['transaction']['gasUsed']))
    list_gasPrice.append(int(swap_list_data[i]['transaction']['gasPrice']))
df_swap.insert(0,'timestamp',list_timestamp)
df_swap.insert(1,'blockNumber',list_blockNumber)
df_swap.insert(2,'gasUsed',list_gasUsed)
df_swap.insert(3,'gasPrice',list_gasPrice)
for i in range(0,len(swap_list_data)):
    df_swap['timestamp'][i] = datetime.datetime.fromtimestamp(df_swap['timestamp'][i])
df_swap.rename(columns={'timestamp':'Date'},inplace=True)
df_swap.to_excel('tx_usdc_eth_500_swap15.xlsx', index = False, header = True)

1627747964
154228


C:\Users\Administrator\AppData\Local\Temp\ipykernel_4404\1391815605.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_swap['timestamp'][i] = datetime.datetime.fromtimestamp(df_swap['timestamp'][i])


In [10]:
swap01 = pd.read_excel('tx_usdc_eth_500_swap01.xlsx')
swap02 = pd.read_excel('tx_usdc_eth_500_swap02.xlsx')
swap03 = pd.read_excel('tx_usdc_eth_500_swap03.xlsx')
swap04 = pd.read_excel('tx_usdc_eth_500_swap04.xlsx')
swap05 = pd.read_excel('tx_usdc_eth_500_swap05.xlsx')
swap06 = pd.read_excel('tx_usdc_eth_500_swap06.xlsx')
swap07 = pd.read_excel('tx_usdc_eth_500_swap07.xlsx')
swap08 = pd.read_excel('tx_usdc_eth_500_swap08.xlsx')
swap09 = pd.read_excel('tx_usdc_eth_500_swap09.xlsx')
swap10 = pd.read_excel('tx_usdc_eth_500_swap10.xlsx')
swap11 = pd.read_excel('tx_usdc_eth_500_swap11.xlsx')
swap12 = pd.read_excel('tx_usdc_eth_500_swap12.xlsx')
swap13 = pd.read_excel('tx_usdc_eth_500_swap13.xlsx')
swap14 = pd.read_excel('tx_usdc_eth_500_swap14.xlsx')
swap15 = pd.read_excel('tx_usdc_eth_500_swap15.xlsx')
df_swap = pd.concat([swap01,swap02,swap03,swap04,swap05,swap06,swap07,swap08,swap09,swap10,swap11,swap12,swap13,swap14,swap15], axis=0).reset_index()
df_swap.drop(df_swap.columns[0],axis=1,inplace=True)
df_swap.to_csv('tx_usdc_eth_500_swaps.csv', index = False, header = True)

In [9]:
df_swap

,Date,blockNumber,gasUsed,gasPrice,amount0,amount1,amountUSD,sqrtPriceX96,tick
0,2022-07-31 23:59:45,15251201,460000,7158514657,2519.688547,-1.471324,2517.250975,1914996634109076168271941236602557,201867
1,2022-07-31 23:59:45,15251201,260326,8658514657,699.957026,-0.408724,699.278172,1914994598062899967289002677858209,201867
2,2022-07-31 23:59:30,15251200,430000,6363801108,2524.138432,-1.473933,2521.706206,1915003963455210883682605926604751,201868
3,2022-07-31 23:58:41,15251197,177405,8045343544,-8340.256475,4.875000,8336.348298,1915011305801558048123548901200871,201868
4,2022-07-31 23:58:08,15251196,248076,7318827918,17125.111058,-10.000000,17108.647962,1914987033304021242869747996992545,201867
...,...,...,...,...,...,...,...,...,...
2089514,2021-08-01 00:18:33,12934350,1000000,0,-130371.906480,52.968750,130280.028355,1596765124594184444326002956802912,198233
2089515,2021-08-01 00:17:58,12934346,286296,500000000000,-69.665683,0.028297,69.606928,1596351644414801049237732528579034,198227
2089516,2021-08-01 00:17:31,12934343,1200000,1,-59286.687655,24.077426,59232.446471,1596351375977074249017908070763885,198227
2089517,2021-08-01 00:17:09,12934340,1200000,1,-24687.333974,10.023953,24662.250715,1596122963605998750519373475185232,198225


## Flashed

In [85]:
query_body = '''
{
    flashes(first:1000,orderBy: timestamp, orderDirection: desc, where:
     { pool: "0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640" }
    ){
    transaction{
    blockNumber
    timestamp
    gasUsed
    gasPrice
    }
    amount0
    amount1
    amountUSD
    amount0Paid
    amount1Paid
    }
    }
'''
query_request = gql(query_body)
flash_origin_data = client.execute(query_request)
print(flash_origin_data)

{'flashes': []}


## Collects

In [ ]:
query_body = '''
{
    collects(first:1000,orderBy: timestamp, orderDirection: desc, where:
     { pool: "0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640" }
    ){
    transaction{
    blockNumber
    timestamp
    gasUsed
    gasPrice
    }
    amount0
    amount1
    amountUSD
    tickLower
    tickUpper
    }
    }
'''
query_request = gql(query_body)
collect_origin_data = client.execute(query_request)
print(collect_origin_data)

## Merge

In [3]:
df_mint = pd.read_csv('tx_usdc_eth_500_mints.csv')
df_burn = pd.read_csv('tx_usdc_eth_500_burns.csv')
df_swap = pd.read_csv('tx_usdc_eth_500_swaps.csv')

In [5]:
type1 = []
type2 = []
type3 = []

for i in range(0,len(df_mint['Date'])):
    type1.append('Mint')
for i in range(0,len(df_burn['Date'])):
    type2.append('Burn')
for i in range(0,len(df_swap['Date'])):
    type3.append('Swap')
    
df_mint['type'] = type1
df_burn['type'] = type2
df_swap['type'] = type3

In [6]:
df = pd.merge(df_mint,df_burn,how='outer')
df = pd.merge(df,df_swap,how='outer')
df = df.sort_values(by=['Date']).reset_index()
df.drop(df.columns[0],axis=1,inplace=True)
df

,Date,blockNumber,gasUsed,gasPrice,amount0,amount1,amountUSD,tickLower,tickUpper,type,sqrtPriceX96,tick
0,2021-08-01 00:12:44,12934312,1200000,1694000000001,835181.684375,-339.293421,833401.758712,NaN,NaN,Swap,1596027870593163232344302683000000,198223.0
1,2021-08-01 00:13:33,12934315,395445,1544000000000,62189.936827,3.641137,71139.230468,198210.0,198320.0,Burn,NaN,NaN
2,2021-08-01 00:17:09,12934340,1200000,1,-24687.333974,10.023953,24662.250715,NaN,NaN,Swap,1596122963605998750519373475185232,198225.0
3,2021-08-01 00:17:31,12934343,1200000,1,-59286.687655,24.077426,59232.446471,NaN,NaN,Swap,1596351375977074249017908070763885,198227.0
4,2021-08-01 00:17:58,12934346,286296,500000000000,-69.665683,0.028297,69.606928,NaN,NaN,Swap,1596351644414801049237732528579034,198227.0
...,...,...,...,...,...,...,...,...,...,...,...,...
2137969,2022-07-31 23:58:41,15251197,177405,8045343544,-8340.256475,4.875000,8336.348298,NaN,NaN,Swap,1915011305801558048123548901200871,201868.0
2137970,2022-07-31 23:59:30,15251200,430000,6363801108,2524.138432,-1.473933,2521.706206,NaN,NaN,Swap,1915003963455210883682605926604751,201868.0
2137971,2022-07-31 23:59:30,15251200,318334,7863801108,0.000000,0.000000,0.000000,201240.0,204200.0,Burn,NaN,NaN
2137972,2022-07-31 23:59:45,15251201,460000,7158514657,2519.688547,-1.471324,2517.250975,NaN,NaN,Swap,1914996634109076168271941236602557,201867.0


In [8]:
df.columns

Index(['Date', 'blockNumber', 'gasUsed', 'gasPrice', 'amount0', 'amount1',
       'amountUSD', 'tickLower', 'tickUpper', 'type', 'sqrtPriceX96', 'tick'],
      dtype='object')

In [9]:
df = df[['Date', 'blockNumber', 'gasUsed', 'gasPrice', 'amount0', 'amount1',
       'amountUSD', 'tickLower', 'tickUpper', 'sqrtPriceX96', 'tick', 'type']]

In [10]:
df

,Date,blockNumber,gasUsed,gasPrice,amount0,amount1,amountUSD,tickLower,tickUpper,sqrtPriceX96,tick,type
0,2021-08-01 00:12:44,12934312,1200000,1694000000001,835181.684375,-339.293421,833401.758712,NaN,NaN,1596027870593163232344302683000000,198223.0,Swap
1,2021-08-01 00:13:33,12934315,395445,1544000000000,62189.936827,3.641137,71139.230468,198210.0,198320.0,NaN,NaN,Burn
2,2021-08-01 00:17:09,12934340,1200000,1,-24687.333974,10.023953,24662.250715,NaN,NaN,1596122963605998750519373475185232,198225.0,Swap
3,2021-08-01 00:17:31,12934343,1200000,1,-59286.687655,24.077426,59232.446471,NaN,NaN,1596351375977074249017908070763885,198227.0,Swap
4,2021-08-01 00:17:58,12934346,286296,500000000000,-69.665683,0.028297,69.606928,NaN,NaN,1596351644414801049237732528579034,198227.0,Swap
...,...,...,...,...,...,...,...,...,...,...,...,...
2137969,2022-07-31 23:58:41,15251197,177405,8045343544,-8340.256475,4.875000,8336.348298,NaN,NaN,1915011305801558048123548901200871,201868.0,Swap
2137970,2022-07-31 23:59:30,15251200,430000,6363801108,2524.138432,-1.473933,2521.706206,NaN,NaN,1915003963455210883682605926604751,201868.0,Swap
2137971,2022-07-31 23:59:30,15251200,318334,7863801108,0.000000,0.000000,0.000000,201240.0,204200.0,NaN,NaN,Burn
2137972,2022-07-31 23:59:45,15251201,460000,7158514657,2519.688547,-1.471324,2517.250975,NaN,NaN,1914996634109076168271941236602557,201867.0,Swap


In [11]:
df.to_csv('tx_usdc_eth_500.csv', index = False, header = True)

# USDC/ETH 0.3% Pool

Fee Tier: 0.3%; Tick Spacing: 60; Token0: USD Coin; Token1: Wrapped Ether.

In [14]:
sample_transport = RequestsHTTPTransport(
    url='https://api.thegraph.com/subgraphs/name/uniswap/uniswap-v3',
    verify=True,
    retries=5,
)
client = Client(
    transport=sample_transport
)
pool = "0x8ad599c3a0ff1de082011efddc58f1908eb6e6d8"

In [15]:
# Mint
# From Fri May 07 2021 00:00:00 GMT+0800 (香港标准时间) To Mon Nov 07 2022 00:00:00 GMT+0800 (香港标准时间)
timestamp_lt = 1667750400 # Mon Nov 07 2022 00:00:00 GMT+0800 (香港标准时间)
timestamp_gt = 1620316800 # Fri May 07 2021 00:00:00 GMT+0800 (香港标准时间)
mint_list_data = []

while timestamp_lt > timestamp_gt:
    for i in range(0,6):
        skip = i * 1000
        query_body = '''
        {
            mints(skip:%i,first:1000,orderBy: timestamp, orderDirection: desc, where:
             { pool:"0x8ad599c3a0ff1de082011efddc58f1908eb6e6d8",timestamp_lt:%i}
            ){
            transaction{
            blockNumber
            timestamp
            gasUsed
            gasPrice
            }
            amount0
            amount1
            amountUSD
            tickLower
            tickUpper
            }
            }
        '''%(skip,timestamp_lt)
        query_request = gql(query_body)
        if skip == 0:
            mint_origin_data = client.execute(query_request)
            mint_origin_data = mint_origin_data['mints']
        else:
            mint_origin_data.extend(client.execute(query_request)['mints'])
    mint_list_data.extend(mint_origin_data)
    timestamp_lt = int(mint_list_data[-1]['transaction']['timestamp'])
    
while int(mint_list_data[-1]['transaction']['timestamp']) < timestamp_gt:
    del mint_list_data[-1]
print(mint_list_data[-1]['transaction']['timestamp'])
print(len(mint_list_data))

df_mint = pd.DataFrame(mint_list_data)
df_mint.drop(df_mint.columns[0],axis=1,inplace=True)
list_blockNumber = []
list_timestamp = []
list_gasUsed = []
list_gasPrice = []
for i in range(0,len(mint_list_data)):
    list_blockNumber.append(int(mint_list_data[i]['transaction']['blockNumber']))
    list_timestamp.append(int(mint_list_data[i]['transaction']['timestamp']))
    list_gasUsed.append(int(mint_list_data[i]['transaction']['gasUsed']))
    list_gasPrice.append(int(mint_list_data[i]['transaction']['gasPrice']))
df_mint.insert(0,'timestamp',list_timestamp)
df_mint.insert(1,'blockNumber',list_blockNumber)
df_mint.insert(2,'gasUsed',list_gasUsed)
df_mint.insert(3,'gasPrice',list_gasPrice)
for i in range(0,len(mint_list_data)):
    df_mint['timestamp'][i] = datetime.datetime.fromtimestamp(df_mint['timestamp'][i])
df_mint.rename(columns={'timestamp':'Date'},inplace=True)
df_mint.to_csv('tx_usdc_eth_3000_mints.csv', index = False, header = True)

1620317986
74761


/var/folders/s0/2c3pyzkd1h31qv62fmmv3cph0000gn/T/ipykernel_1817/1648815881.py:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_mint['timestamp'][i] = datetime.datetime.fromtimestamp(df_mint['timestamp'][i])


In [16]:
timestamp_lt = 1667750400 # Mon Nov 07 2022 00:00:00 GMT+0800 (香港标准时间)
timestamp_gt = 1620316800 # Fri May 07 2021 00:00:00 GMT+0800 (香港标准时间)
burn_list_data = []
# Burn
while timestamp_lt > timestamp_gt:
    for i in range(0,6):
        skip = i * 1000
        query_body = '''
        {
            burns(skip:%i,first:1000,orderBy: timestamp, orderDirection: desc, where:
             { pool:"0x8ad599c3a0ff1de082011efddc58f1908eb6e6d8",timestamp_lt:%i}
            ){
            transaction{
            blockNumber
            timestamp
            gasUsed
            gasPrice
            }
            amount0
            amount1
            amountUSD
            tickLower
            tickUpper
            }
            }
        '''%(skip,timestamp_lt)
        query_request = gql(query_body)
        if skip == 0:
            burn_origin_data = client.execute(query_request)
            burn_origin_data = burn_origin_data['burns']
        else:
            burn_origin_data.extend(client.execute(query_request)['burns'])
    burn_list_data.extend(burn_origin_data)
    timestamp_lt = int(burn_list_data[-1]['transaction']['timestamp'])

while int(burn_list_data[-1]['transaction']['timestamp']) < timestamp_gt:
    del burn_list_data[-1]

print(burn_list_data[-1]['transaction']['timestamp'])
print(len(burn_list_data))

df_burn = pd.DataFrame(burn_list_data)
df_burn.drop(df_burn.columns[0],axis=1,inplace=True)
list_blockNumber = []
list_timestamp = []
list_gasUsed = []
list_gasPrice = []
for i in range(0,len(burn_list_data)):
    list_blockNumber.append(int(burn_list_data[i]['transaction']['blockNumber']))
    list_timestamp.append(int(burn_list_data[i]['transaction']['timestamp']))
    list_gasUsed.append(int(burn_list_data[i]['transaction']['gasUsed']))
    list_gasPrice.append(int(burn_list_data[i]['transaction']['gasPrice']))
df_burn.insert(0,'timestamp',list_timestamp)
df_burn.insert(1,'blockNumber',list_blockNumber)
df_burn.insert(2,'gasUsed',list_gasUsed)
df_burn.insert(3,'gasPrice',list_gasPrice)
for i in range(0,len(burn_list_data)):
    df_burn['timestamp'][i] = datetime.datetime.fromtimestamp(df_burn['timestamp'][i])
df_burn.rename(columns={'timestamp':'Date'},inplace=True)
df_burn.to_csv('tx_usdc_eth_3000_burns.csv', index = False, header = True)

1620318451
70744


/var/folders/s0/2c3pyzkd1h31qv62fmmv3cph0000gn/T/ipykernel_1817/369835898.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_burn['timestamp'][i] = datetime.datetime.fromtimestamp(df_burn['timestamp'][i])


In [18]:
# Swap
# From Fri May 07 2021 00:00:00 GMT+0800 (香港标准时间) To Mon Nov 07 2022 00:00:00 GMT+0800 (香港标准时间)
time_list = list(np.linspace(1667750400,1620316800,10))
print(time_list)

[1667750400.0, 1662480000.0, 1657209600.0, 1651939200.0, 1646668800.0, 1641398400.0, 1636128000.0, 1630857600.0, 1625587200.0, 1620316800.0]


In [19]:
timestamp_lt = time_list[0]
timestamp_gt = time_list[1]
swap_list_data = []
while timestamp_lt > timestamp_gt:
    for i in range(0,6):
        skip = i * 1000
        query_body = '''
        {
            swaps(skip:%i,first:1000,orderBy: timestamp, orderDirection: desc, where:
             { pool:"0x8ad599c3a0ff1de082011efddc58f1908eb6e6d8",timestamp_lt:%i}
            ){
            transaction{
            blockNumber
            timestamp
            gasUsed
            gasPrice
            }
            amount0
            amount1
            amountUSD
            sqrtPriceX96
            tick
            }
            }
        '''%(skip,timestamp_lt)
        query_request = gql(query_body)
        if skip == 0:
            swap_origin_data = client.execute(query_request)
            swap_origin_data = swap_origin_data['swaps']
        else:
            swap_origin_data.extend(client.execute(query_request)['swaps'])
    swap_list_data.extend(swap_origin_data)
    timestamp_lt = int(swap_list_data[-1]['transaction']['timestamp'])

while int(swap_list_data[-1]['transaction']['timestamp']) < timestamp_gt:
    del swap_list_data[-1]


print(swap_list_data[-1]['transaction']['timestamp'])
print(len(swap_list_data))

df_swap = pd.DataFrame(swap_list_data)
df_swap.drop(df_swap.columns[0],axis=1,inplace=True)
list_blockNumber = []
list_timestamp = []
list_gasUsed = []
list_gasPrice = []
for i in range(0,len(swap_list_data)):
    list_blockNumber.append(int(swap_list_data[i]['transaction']['blockNumber']))
    list_timestamp.append(int(swap_list_data[i]['transaction']['timestamp']))
    list_gasUsed.append(int(swap_list_data[i]['transaction']['gasUsed']))
    list_gasPrice.append(int(swap_list_data[i]['transaction']['gasPrice']))
df_swap.insert(0,'timestamp',list_timestamp)
df_swap.insert(1,'blockNumber',list_blockNumber)
df_swap.insert(2,'gasUsed',list_gasUsed)
df_swap.insert(3,'gasPrice',list_gasPrice)
for i in range(0,len(swap_list_data)):
    df_swap['timestamp'][i] = datetime.datetime.fromtimestamp(df_swap['timestamp'][i])
df_swap.rename(columns={'timestamp':'Date'},inplace=True)
df_swap.to_csv('tx_usdc_eth_3000_swap01.csv', index = False, header = True)

1662480294
38590


/var/folders/s0/2c3pyzkd1h31qv62fmmv3cph0000gn/T/ipykernel_1817/1852239054.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_swap['timestamp'][i] = datetime.datetime.fromtimestamp(df_swap['timestamp'][i])


In [20]:
timestamp_lt = time_list[1]
timestamp_gt = time_list[2]
swap_list_data = []
while timestamp_lt > timestamp_gt:
    for i in range(0,6):
        skip = i * 1000
        query_body = '''
        {
            swaps(skip:%i,first:1000,orderBy: timestamp, orderDirection: desc, where:
             { pool:"0x8ad599c3a0ff1de082011efddc58f1908eb6e6d8",timestamp_lt:%i}
            ){
            transaction{
            blockNumber
            timestamp
            gasUsed
            gasPrice
            }
            amount0
            amount1
            amountUSD
            sqrtPriceX96
            tick
            }
            }
        '''%(skip,timestamp_lt)
        query_request = gql(query_body)
        if skip == 0:
            swap_origin_data = client.execute(query_request)
            swap_origin_data = swap_origin_data['swaps']
        else:
            swap_origin_data.extend(client.execute(query_request)['swaps'])
    swap_list_data.extend(swap_origin_data)
    timestamp_lt = int(swap_list_data[-1]['transaction']['timestamp'])

while int(swap_list_data[-1]['transaction']['timestamp']) < timestamp_gt:
    del swap_list_data[-1]

print(swap_list_data[-1]['transaction']['timestamp'])
print(len(swap_list_data))

df_swap = pd.DataFrame(swap_list_data)
df_swap.drop(df_swap.columns[0],axis=1,inplace=True)
list_blockNumber = []
list_timestamp = []
list_gasUsed = []
list_gasPrice = []
for i in range(0,len(swap_list_data)):
    list_blockNumber.append(int(swap_list_data[i]['transaction']['blockNumber']))
    list_timestamp.append(int(swap_list_data[i]['transaction']['timestamp']))
    list_gasUsed.append(int(swap_list_data[i]['transaction']['gasUsed']))
    list_gasPrice.append(int(swap_list_data[i]['transaction']['gasPrice']))
df_swap.insert(0,'timestamp',list_timestamp)
df_swap.insert(1,'blockNumber',list_blockNumber)
df_swap.insert(2,'gasUsed',list_gasUsed)
df_swap.insert(3,'gasPrice',list_gasPrice)
for i in range(0,len(swap_list_data)):
    df_swap['timestamp'][i] = datetime.datetime.fromtimestamp(df_swap['timestamp'][i])
df_swap.rename(columns={'timestamp':'Date'},inplace=True)
df_swap.to_csv('tx_usdc_eth_3000_swap02.csv', index = False, header = True)

1657209761
41195


/var/folders/s0/2c3pyzkd1h31qv62fmmv3cph0000gn/T/ipykernel_1817/398430130.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_swap['timestamp'][i] = datetime.datetime.fromtimestamp(df_swap['timestamp'][i])


In [21]:
timestamp_lt = time_list[2]
timestamp_gt = time_list[3]
swap_list_data = []
while timestamp_lt > timestamp_gt:
    for i in range(0,6):
        skip = i * 1000
        query_body = '''
        {
            swaps(skip:%i,first:1000,orderBy: timestamp, orderDirection: desc, where:
             { pool:"0x8ad599c3a0ff1de082011efddc58f1908eb6e6d8",timestamp_lt:%i}
            ){
            transaction{
            blockNumber
            timestamp
            gasUsed
            gasPrice
            }
            amount0
            amount1
            amountUSD
            sqrtPriceX96
            tick
            }
            }
        '''%(skip,timestamp_lt)
        query_request = gql(query_body)
        if skip == 0:
            swap_origin_data = client.execute(query_request)
            swap_origin_data = swap_origin_data['swaps']
        else:
            swap_origin_data.extend(client.execute(query_request)['swaps'])
    swap_list_data.extend(swap_origin_data)
    timestamp_lt = int(swap_list_data[-1]['transaction']['timestamp'])

while int(swap_list_data[-1]['transaction']['timestamp']) < timestamp_gt:
    del swap_list_data[-1]

print(swap_list_data[-1]['transaction']['timestamp'])
print(len(swap_list_data))

df_swap = pd.DataFrame(swap_list_data)
df_swap.drop(df_swap.columns[0],axis=1,inplace=True)
list_blockNumber = []
list_timestamp = []
list_gasUsed = []
list_gasPrice = []
for i in range(0,len(swap_list_data)):
    list_blockNumber.append(int(swap_list_data[i]['transaction']['blockNumber']))
    list_timestamp.append(int(swap_list_data[i]['transaction']['timestamp']))
    list_gasUsed.append(int(swap_list_data[i]['transaction']['gasUsed']))
    list_gasPrice.append(int(swap_list_data[i]['transaction']['gasPrice']))
df_swap.insert(0,'timestamp',list_timestamp)
df_swap.insert(1,'blockNumber',list_blockNumber)
df_swap.insert(2,'gasUsed',list_gasUsed)
df_swap.insert(3,'gasPrice',list_gasPrice)
for i in range(0,len(swap_list_data)):
    df_swap['timestamp'][i] = datetime.datetime.fromtimestamp(df_swap['timestamp'][i])
df_swap.rename(columns={'timestamp':'Date'},inplace=True)
df_swap.to_csv('tx_usdc_eth_3000_swap03.csv', index = False, header = True)

1651942227
46576


/var/folders/s0/2c3pyzkd1h31qv62fmmv3cph0000gn/T/ipykernel_1817/3144210975.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_swap['timestamp'][i] = datetime.datetime.fromtimestamp(df_swap['timestamp'][i])


In [22]:
timestamp_lt = time_list[3]
timestamp_gt = time_list[4]
swap_list_data = []
while timestamp_lt > timestamp_gt:
    for i in range(0,6):
        skip = i * 1000
        query_body = '''
        {
            swaps(skip:%i,first:1000,orderBy: timestamp, orderDirection: desc, where:
             { pool:"0x8ad599c3a0ff1de082011efddc58f1908eb6e6d8",timestamp_lt:%i}
            ){
            transaction{
            blockNumber
            timestamp
            gasUsed
            gasPrice
            }
            amount0
            amount1
            amountUSD
            sqrtPriceX96
            tick
            }
            }
        '''%(skip,timestamp_lt)
        query_request = gql(query_body)
        if skip == 0:
            swap_origin_data = client.execute(query_request)
            swap_origin_data = swap_origin_data['swaps']
        else:
            swap_origin_data.extend(client.execute(query_request)['swaps'])
    swap_list_data.extend(swap_origin_data)
    timestamp_lt = int(swap_list_data[-1]['transaction']['timestamp'])

while int(swap_list_data[-1]['transaction']['timestamp']) < timestamp_gt:
    del swap_list_data[-1]

print(swap_list_data[-1]['transaction']['timestamp'])
print(len(swap_list_data))

df_swap = pd.DataFrame(swap_list_data)
df_swap.drop(df_swap.columns[0],axis=1,inplace=True)
list_blockNumber = []
list_timestamp = []
list_gasUsed = []
list_gasPrice = []
for i in range(0,len(swap_list_data)):
    list_blockNumber.append(int(swap_list_data[i]['transaction']['blockNumber']))
    list_timestamp.append(int(swap_list_data[i]['transaction']['timestamp']))
    list_gasUsed.append(int(swap_list_data[i]['transaction']['gasUsed']))
    list_gasPrice.append(int(swap_list_data[i]['transaction']['gasPrice']))
df_swap.insert(0,'timestamp',list_timestamp)
df_swap.insert(1,'blockNumber',list_blockNumber)
df_swap.insert(2,'gasUsed',list_gasUsed)
df_swap.insert(3,'gasPrice',list_gasPrice)
for i in range(0,len(swap_list_data)):
    df_swap['timestamp'][i] = datetime.datetime.fromtimestamp(df_swap['timestamp'][i])
df_swap.rename(columns={'timestamp':'Date'},inplace=True)
df_swap.to_csv('tx_usdc_eth_3000_swap04.csv', index = False, header = True)

1646668921
22899


/var/folders/s0/2c3pyzkd1h31qv62fmmv3cph0000gn/T/ipykernel_1817/1733569331.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_swap['timestamp'][i] = datetime.datetime.fromtimestamp(df_swap['timestamp'][i])


In [23]:
timestamp_lt = time_list[4]
timestamp_gt = time_list[5]
swap_list_data = []
while timestamp_lt > timestamp_gt:
    for i in range(0,6):
        skip = i * 1000
        query_body = '''
        {
            swaps(skip:%i,first:1000,orderBy: timestamp, orderDirection: desc, where:
             { pool:"0x8ad599c3a0ff1de082011efddc58f1908eb6e6d8",timestamp_lt:%i}
            ){
            transaction{
            blockNumber
            timestamp
            gasUsed
            gasPrice
            }
            amount0
            amount1
            amountUSD
            sqrtPriceX96
            tick
            }
            }
        '''%(skip,timestamp_lt)
        query_request = gql(query_body)
        if skip == 0:
            swap_origin_data = client.execute(query_request)
            swap_origin_data = swap_origin_data['swaps']
        else:
            swap_origin_data.extend(client.execute(query_request)['swaps'])
    swap_list_data.extend(swap_origin_data)
    timestamp_lt = int(swap_list_data[-1]['transaction']['timestamp'])

while int(swap_list_data[-1]['transaction']['timestamp']) < timestamp_gt:
    del swap_list_data[-1]

print(swap_list_data[-1]['transaction']['timestamp'])
print(len(swap_list_data))

df_swap = pd.DataFrame(swap_list_data)
df_swap.drop(df_swap.columns[0],axis=1,inplace=True)
list_blockNumber = []
list_timestamp = []
list_gasUsed = []
list_gasPrice = []
for i in range(0,len(swap_list_data)):
    list_blockNumber.append(int(swap_list_data[i]['transaction']['blockNumber']))
    list_timestamp.append(int(swap_list_data[i]['transaction']['timestamp']))
    list_gasUsed.append(int(swap_list_data[i]['transaction']['gasUsed']))
    list_gasPrice.append(int(swap_list_data[i]['transaction']['gasPrice']))
df_swap.insert(0,'timestamp',list_timestamp)
df_swap.insert(1,'blockNumber',list_blockNumber)
df_swap.insert(2,'gasUsed',list_gasUsed)
df_swap.insert(3,'gasPrice',list_gasPrice)
for i in range(0,len(swap_list_data)):
    df_swap['timestamp'][i] = datetime.datetime.fromtimestamp(df_swap['timestamp'][i])
df_swap.rename(columns={'timestamp':'Date'},inplace=True)
df_swap.to_csv('tx_usdc_eth_3000_swap05.csv', index = False, header = True)

1641398555
28880


/var/folders/s0/2c3pyzkd1h31qv62fmmv3cph0000gn/T/ipykernel_1817/3958303090.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_swap['timestamp'][i] = datetime.datetime.fromtimestamp(df_swap['timestamp'][i])


In [24]:
timestamp_lt = time_list[5]
timestamp_gt = time_list[6]
swap_list_data = []
while timestamp_lt > timestamp_gt:
    for i in range(0,6):
        skip = i * 1000
        query_body = '''
        {
            swaps(skip:%i,first:1000,orderBy: timestamp, orderDirection: desc, where:
             { pool:"0x8ad599c3a0ff1de082011efddc58f1908eb6e6d8",timestamp_lt:%i}
            ){
            transaction{
            blockNumber
            timestamp
            gasUsed
            gasPrice
            }
            amount0
            amount1
            amountUSD
            sqrtPriceX96
            tick
            }
            }
        '''%(skip,timestamp_lt)
        query_request = gql(query_body)
        if skip == 0:
            swap_origin_data = client.execute(query_request)
            swap_origin_data = swap_origin_data['swaps']
        else:
            swap_origin_data.extend(client.execute(query_request)['swaps'])
    swap_list_data.extend(swap_origin_data)
    timestamp_lt = int(swap_list_data[-1]['transaction']['timestamp'])

while int(swap_list_data[-1]['transaction']['timestamp']) < timestamp_gt:
    del swap_list_data[-1]

print(swap_list_data[-1]['transaction']['timestamp'])
print(len(swap_list_data))

df_swap = pd.DataFrame(swap_list_data)
df_swap.drop(df_swap.columns[0],axis=1,inplace=True)
list_blockNumber = []
list_timestamp = []
list_gasUsed = []
list_gasPrice = []
for i in range(0,len(swap_list_data)):
    list_blockNumber.append(int(swap_list_data[i]['transaction']['blockNumber']))
    list_timestamp.append(int(swap_list_data[i]['transaction']['timestamp']))
    list_gasUsed.append(int(swap_list_data[i]['transaction']['gasUsed']))
    list_gasPrice.append(int(swap_list_data[i]['transaction']['gasPrice']))
df_swap.insert(0,'timestamp',list_timestamp)
df_swap.insert(1,'blockNumber',list_blockNumber)
df_swap.insert(2,'gasUsed',list_gasUsed)
df_swap.insert(3,'gasPrice',list_gasPrice)
for i in range(0,len(swap_list_data)):
    df_swap['timestamp'][i] = datetime.datetime.fromtimestamp(df_swap['timestamp'][i])
df_swap.rename(columns={'timestamp':'Date'},inplace=True)
df_swap.to_csv('tx_usdc_eth_3000_swap06.csv', index = False, header = True)

1636129242
26587


/var/folders/s0/2c3pyzkd1h31qv62fmmv3cph0000gn/T/ipykernel_1817/2176230466.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_swap['timestamp'][i] = datetime.datetime.fromtimestamp(df_swap['timestamp'][i])


In [25]:
timestamp_lt = time_list[6]
timestamp_gt = time_list[7]
swap_list_data = []
while timestamp_lt > timestamp_gt:
    for i in range(0,6):
        skip = i * 1000
        query_body = '''
        {
            swaps(skip:%i,first:1000,orderBy: timestamp, orderDirection: desc, where:
             { pool:"0x8ad599c3a0ff1de082011efddc58f1908eb6e6d8",timestamp_lt:%i}
            ){
            transaction{
            blockNumber
            timestamp
            gasUsed
            gasPrice
            }
            amount0
            amount1
            amountUSD
            sqrtPriceX96
            tick
            }
            }
        '''%(skip,timestamp_lt)
        query_request = gql(query_body)
        if skip == 0:
            swap_origin_data = client.execute(query_request)
            swap_origin_data = swap_origin_data['swaps']
        else:
            swap_origin_data.extend(client.execute(query_request)['swaps'])
    swap_list_data.extend(swap_origin_data)
    timestamp_lt = int(swap_list_data[-1]['transaction']['timestamp'])

while int(swap_list_data[-1]['transaction']['timestamp']) < timestamp_gt:
    del swap_list_data[-1]

print(swap_list_data[-1]['transaction']['timestamp'])
print(len(swap_list_data))

df_swap = pd.DataFrame(swap_list_data)
df_swap.drop(df_swap.columns[0],axis=1,inplace=True)
list_blockNumber = []
list_timestamp = []
list_gasUsed = []
list_gasPrice = []
for i in range(0,len(swap_list_data)):
    list_blockNumber.append(int(swap_list_data[i]['transaction']['blockNumber']))
    list_timestamp.append(int(swap_list_data[i]['transaction']['timestamp']))
    list_gasUsed.append(int(swap_list_data[i]['transaction']['gasUsed']))
    list_gasPrice.append(int(swap_list_data[i]['transaction']['gasPrice']))
df_swap.insert(0,'timestamp',list_timestamp)
df_swap.insert(1,'blockNumber',list_blockNumber)
df_swap.insert(2,'gasUsed',list_gasUsed)
df_swap.insert(3,'gasPrice',list_gasPrice)
for i in range(0,len(swap_list_data)):
    df_swap['timestamp'][i] = datetime.datetime.fromtimestamp(df_swap['timestamp'][i])
df_swap.rename(columns={'timestamp':'Date'},inplace=True)
df_swap.to_csv('tx_usdc_eth_3000_swap07.csv', index = False, header = True)

1630861635
29749


/var/folders/s0/2c3pyzkd1h31qv62fmmv3cph0000gn/T/ipykernel_1817/1975555565.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_swap['timestamp'][i] = datetime.datetime.fromtimestamp(df_swap['timestamp'][i])


In [27]:
timestamp_lt = time_list[7]
timestamp_gt = time_list[8]
swap_list_data = []
while timestamp_lt > timestamp_gt:
    for i in range(0,6):
        skip = i * 1000
        query_body = '''
        {
            swaps(skip:%i,first:1000,orderBy: timestamp, orderDirection: desc, where:
             { pool:"0x8ad599c3a0ff1de082011efddc58f1908eb6e6d8",timestamp_lt:%i}
            ){
            transaction{
            blockNumber
            timestamp
            gasUsed
            gasPrice
            }
            amount0
            amount1
            amountUSD
            sqrtPriceX96
            tick
            }
            }
        '''%(skip,timestamp_lt)
        query_request = gql(query_body)
        if skip == 0:
            swap_origin_data = client.execute(query_request)
            swap_origin_data = swap_origin_data['swaps']
        else:
            swap_origin_data.extend(client.execute(query_request)['swaps'])
    swap_list_data.extend(swap_origin_data)
    timestamp_lt = int(swap_list_data[-1]['transaction']['timestamp'])

while int(swap_list_data[-1]['transaction']['timestamp']) < timestamp_gt:
    del swap_list_data[-1]

print(swap_list_data[-1]['transaction']['timestamp'])
print(len(swap_list_data))

df_swap = pd.DataFrame(swap_list_data)
df_swap.drop(df_swap.columns[0],axis=1,inplace=True)
list_blockNumber = []
list_timestamp = []
list_gasUsed = []
list_gasPrice = []
for i in range(0,len(swap_list_data)):
    list_blockNumber.append(int(swap_list_data[i]['transaction']['blockNumber']))
    list_timestamp.append(int(swap_list_data[i]['transaction']['timestamp']))
    list_gasUsed.append(int(swap_list_data[i]['transaction']['gasUsed']))
    list_gasPrice.append(int(swap_list_data[i]['transaction']['gasPrice']))
df_swap.insert(0,'timestamp',list_timestamp)
df_swap.insert(1,'blockNumber',list_blockNumber)
df_swap.insert(2,'gasUsed',list_gasUsed)
df_swap.insert(3,'gasPrice',list_gasPrice)
for i in range(0,len(swap_list_data)):
    df_swap['timestamp'][i] = datetime.datetime.fromtimestamp(df_swap['timestamp'][i])
df_swap.rename(columns={'timestamp':'Date'},inplace=True)
df_swap.to_csv('tx_usdc_eth_3000_swap08.csv', index = False, header = True)

1625587273
37014


/var/folders/s0/2c3pyzkd1h31qv62fmmv3cph0000gn/T/ipykernel_1817/1832481445.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_swap['timestamp'][i] = datetime.datetime.fromtimestamp(df_swap['timestamp'][i])


In [31]:
timestamp_lt = time_list[8]
timestamp_gt = time_list[8]+(time_list[9]-time_list[8])/2
swap_list_data = []
while timestamp_lt > timestamp_gt:
    for i in range(0,6):
        skip = i * 1000
        query_body = '''
        {
            swaps(skip:%i,first:1000,orderBy: timestamp, orderDirection: desc, where:
             { pool:"0x8ad599c3a0ff1de082011efddc58f1908eb6e6d8",timestamp_lt:%i}
            ){
            transaction{
            blockNumber
            timestamp
            gasUsed
            gasPrice
            }
            amount0
            amount1
            amountUSD
            sqrtPriceX96
            tick
            }
            }
        '''%(skip,timestamp_lt)
        query_request = gql(query_body)
        if skip == 0:
            swap_origin_data = client.execute(query_request)
            swap_origin_data = swap_origin_data['swaps']
        else:
            swap_origin_data.extend(client.execute(query_request)['swaps'])
    swap_list_data.extend(swap_origin_data)
    timestamp_lt = int(swap_list_data[-1]['transaction']['timestamp'])

while int(swap_list_data[-1]['transaction']['timestamp']) < timestamp_gt:
    del swap_list_data[-1]

print(swap_list_data[-1]['transaction']['timestamp'])
print(len(swap_list_data))

df_swap = pd.DataFrame(swap_list_data)
df_swap.drop(df_swap.columns[0],axis=1,inplace=True)
list_blockNumber = []
list_timestamp = []
list_gasUsed = []
list_gasPrice = []
for i in range(0,len(swap_list_data)):
    list_blockNumber.append(int(swap_list_data[i]['transaction']['blockNumber']))
    list_timestamp.append(int(swap_list_data[i]['transaction']['timestamp']))
    list_gasUsed.append(int(swap_list_data[i]['transaction']['gasUsed']))
    list_gasPrice.append(int(swap_list_data[i]['transaction']['gasPrice']))
df_swap.insert(0,'timestamp',list_timestamp)
df_swap.insert(1,'blockNumber',list_blockNumber)
df_swap.insert(2,'gasUsed',list_gasUsed)
df_swap.insert(3,'gasPrice',list_gasPrice)
for i in range(0,len(swap_list_data)):
    df_swap['timestamp'][i] = datetime.datetime.fromtimestamp(df_swap['timestamp'][i])
df_swap.rename(columns={'timestamp':'Date'},inplace=True)
df_swap.to_csv('tx_usdc_eth_3000_swap09.csv', index = False, header = True)

1622952029
31811


/var/folders/s0/2c3pyzkd1h31qv62fmmv3cph0000gn/T/ipykernel_1817/1045401599.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_swap['timestamp'][i] = datetime.datetime.fromtimestamp(df_swap['timestamp'][i])


In [32]:
timestamp_lt = time_list[8]+(time_list[9]-time_list[8])/2
timestamp_gt = time_list[9]
swap_list_data = []
while timestamp_lt > timestamp_gt:
    for i in range(0,6):
        skip = i * 1000
        query_body = '''
        {
            swaps(skip:%i,first:1000,orderBy: timestamp, orderDirection: desc, where:
             { pool:"0x8ad599c3a0ff1de082011efddc58f1908eb6e6d8",timestamp_lt:%i}
            ){
            transaction{
            blockNumber
            timestamp
            gasUsed
            gasPrice
            }
            amount0
            amount1
            amountUSD
            sqrtPriceX96
            tick
            }
            }
        '''%(skip,timestamp_lt)
        query_request = gql(query_body)
        if skip == 0:
            swap_origin_data = client.execute(query_request)
            swap_origin_data = swap_origin_data['swaps']
        else:
            swap_origin_data.extend(client.execute(query_request)['swaps'])
    swap_list_data.extend(swap_origin_data)
    timestamp_lt = int(swap_list_data[-1]['transaction']['timestamp'])

while int(swap_list_data[-1]['transaction']['timestamp']) < timestamp_gt:
    del swap_list_data[-1]

print(swap_list_data[-1]['transaction']['timestamp'])
print(len(swap_list_data))

df_swap = pd.DataFrame(swap_list_data)
df_swap.drop(df_swap.columns[0],axis=1,inplace=True)
list_blockNumber = []
list_timestamp = []
list_gasUsed = []
list_gasPrice = []
for i in range(0,len(swap_list_data)):
    list_blockNumber.append(int(swap_list_data[i]['transaction']['blockNumber']))
    list_timestamp.append(int(swap_list_data[i]['transaction']['timestamp']))
    list_gasUsed.append(int(swap_list_data[i]['transaction']['gasUsed']))
    list_gasPrice.append(int(swap_list_data[i]['transaction']['gasPrice']))
df_swap.insert(0,'timestamp',list_timestamp)
df_swap.insert(1,'blockNumber',list_blockNumber)
df_swap.insert(2,'gasUsed',list_gasUsed)
df_swap.insert(3,'gasPrice',list_gasPrice)
for i in range(0,len(swap_list_data)):
    df_swap['timestamp'][i] = datetime.datetime.fromtimestamp(df_swap['timestamp'][i])
df_swap.rename(columns={'timestamp':'Date'},inplace=True)
df_swap.to_csv('tx_usdc_eth_3000_swap10.csv', index = False, header = True)

1620316802
55449


/var/folders/s0/2c3pyzkd1h31qv62fmmv3cph0000gn/T/ipykernel_1817/3235221715.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_swap['timestamp'][i] = datetime.datetime.fromtimestamp(df_swap['timestamp'][i])


In [33]:
swap01 = pd.read_csv('tx_usdc_eth_3000_swap01.csv')
swap02 = pd.read_csv('tx_usdc_eth_3000_swap02.csv')
swap03 = pd.read_csv('tx_usdc_eth_3000_swap03.csv')
swap04 = pd.read_csv('tx_usdc_eth_3000_swap04.csv')
swap05 = pd.read_csv('tx_usdc_eth_3000_swap05.csv')
swap06 = pd.read_csv('tx_usdc_eth_3000_swap06.csv')
swap07 = pd.read_csv('tx_usdc_eth_3000_swap07.csv')
swap08 = pd.read_csv('tx_usdc_eth_3000_swap08.csv')
swap09 = pd.read_csv('tx_usdc_eth_3000_swap09.csv')
swap10 = pd.read_csv('tx_usdc_eth_3000_swap10.csv')
df_swap = pd.concat([swap01,swap02,swap03,swap04,swap05,swap06,swap07,swap08,swap09,swap10], axis=0).reset_index()
df_swap.drop(df_swap.columns[0],axis=1,inplace=True)
df_swap.to_csv('tx_usdc_eth_3000_swaps.csv', index = False, header = True)

In [17]:
query_body = '''
{
    flashes(first:1000,orderBy: timestamp, orderDirection: desc, where:
     { pool: "0x8ad599c3a0ff1de082011efddc58f1908eb6e6d8" }
    ){
    transaction{
    blockNumber
    timestamp
    gasUsed
    gasPrice
    }
    amount0
    amount1
    amountUSD
    amount0Paid
    amount1Paid
    }
    }
'''
query_request = gql(query_body)
flash_origin_data = client.execute(query_request)
print(flash_origin_data)

{'flashes': []}


In [18]:
query_body = '''
{
    collects(first:1000,orderBy: timestamp, orderDirection: desc, where:
     { pool: "0x8ad599c3a0ff1de082011efddc58f1908eb6e6d8" }
    ){
    transaction{
    blockNumber
    timestamp
    gasUsed
    gasPrice
    }
    amount0
    amount1
    amountUSD
    tickLower
    tickUpper
    }
    }
'''
query_request = gql(query_body)
collect_origin_data = client.execute(query_request)
print(collect_origin_data)

{'collects': []}


In [34]:
df_mint = pd.read_csv('tx_usdc_eth_3000_mints.csv')
df_burn = pd.read_csv('tx_usdc_eth_3000_burns.csv')
df_swap = pd.read_csv('tx_usdc_eth_3000_swaps.csv')

In [35]:
type1 = []
type2 = []
type3 = []

for i in range(0,len(df_mint['Date'])):
    type1.append('Mint')
for i in range(0,len(df_burn['Date'])):
    type2.append('Burn')
for i in range(0,len(df_swap['Date'])):
    type3.append('Swap')
    
df_mint['type'] = type1
df_burn['type'] = type2
df_swap['type'] = type3

In [36]:
df = pd.merge(df_mint,df_burn,how='outer')
df = pd.merge(df,df_swap,how='outer')
df = df.sort_values(by=['Date']).reset_index()
df.drop(df.columns[0],axis=1,inplace=True)
df

,Date,blockNumber,gasUsed,gasPrice,amount0,amount1,amountUSD,tickLower,tickUpper,type,sqrtPriceX96,tick
0,2021-05-07 00:00:02,12381690,264217,72000000000,-9976.568329,2.827993,9991.845481,NaN,NaN,Swap,1331948317419339742814657373906728,194606.0
1,2021-05-07 00:00:08,12381691,295468,71000000000,25006.639122,-7.045440,24967.465270,NaN,NaN,Swap,1331770533611761750547317926373660,194603.0
2,2021-05-07 00:01:31,12381697,2500000,114000000000,342445.806886,-96.291900,341619.189935,NaN,NaN,Swap,1329308189508690444011137977873678,194566.0
3,2021-05-07 00:03:40,12381701,584850,103022499998,42928.544249,-12.045656,42859.074507,NaN,NaN,Swap,1328992823290554149692791604340119,194561.0
4,2021-05-07 00:03:56,12381703,584962,103022499998,77280.814639,-21.670429,77148.447184,NaN,NaN,Swap,1328425471781686714022334313251194,194553.0
...,...,...,...,...,...,...,...,...,...,...,...,...
504250,2022-11-06 23:54:35,15912004,406562,13627036395,-194.837631,0.120000,195.130803,NaN,NaN,Swap,1963276391095642212712933732301232,202365.0
504251,2022-11-06 23:56:11,15912012,429855,12138262168,-373.438395,0.230000,374.000370,NaN,NaN,Swap,1963277768917783198565469977451087,202366.0
504252,2022-11-06 23:56:23,15912013,385376,14005477501,-162.364356,0.100000,162.608660,NaN,NaN,Swap,1963278367970887975023094431864067,202366.0
504253,2022-11-06 23:58:47,15912025,419451,12695226985,-81.182141,0.050000,81.304287,NaN,NaN,Swap,1963278667497440363251906659070557,202366.0


In [37]:
df.columns

Index(['Date', 'blockNumber', 'gasUsed', 'gasPrice', 'amount0', 'amount1',
       'amountUSD', 'tickLower', 'tickUpper', 'type', 'sqrtPriceX96', 'tick'],
      dtype='object')

In [38]:
df = df[['Date', 'blockNumber', 'gasUsed', 'gasPrice', 'amount0', 'amount1',
       'amountUSD', 'tickLower', 'tickUpper', 'sqrtPriceX96', 'tick', 'type']]
df

,Date,blockNumber,gasUsed,gasPrice,amount0,amount1,amountUSD,tickLower,tickUpper,sqrtPriceX96,tick,type
0,2021-05-07 00:00:02,12381690,264217,72000000000,-9976.568329,2.827993,9991.845481,NaN,NaN,1331948317419339742814657373906728,194606.0,Swap
1,2021-05-07 00:00:08,12381691,295468,71000000000,25006.639122,-7.045440,24967.465270,NaN,NaN,1331770533611761750547317926373660,194603.0,Swap
2,2021-05-07 00:01:31,12381697,2500000,114000000000,342445.806886,-96.291900,341619.189935,NaN,NaN,1329308189508690444011137977873678,194566.0,Swap
3,2021-05-07 00:03:40,12381701,584850,103022499998,42928.544249,-12.045656,42859.074507,NaN,NaN,1328992823290554149692791604340119,194561.0,Swap
4,2021-05-07 00:03:56,12381703,584962,103022499998,77280.814639,-21.670429,77148.447184,NaN,NaN,1328425471781686714022334313251194,194553.0,Swap
...,...,...,...,...,...,...,...,...,...,...,...,...
504250,2022-11-06 23:54:35,15912004,406562,13627036395,-194.837631,0.120000,195.130803,NaN,NaN,1963276391095642212712933732301232,202365.0,Swap
504251,2022-11-06 23:56:11,15912012,429855,12138262168,-373.438395,0.230000,374.000370,NaN,NaN,1963277768917783198565469977451087,202366.0,Swap
504252,2022-11-06 23:56:23,15912013,385376,14005477501,-162.364356,0.100000,162.608660,NaN,NaN,1963278367970887975023094431864067,202366.0,Swap
504253,2022-11-06 23:58:47,15912025,419451,12695226985,-81.182141,0.050000,81.304287,NaN,NaN,1963278667497440363251906659070557,202366.0,Swap


In [39]:
df.to_csv('tx_usdc_eth_3000.csv', index = False, header = True)

# USDC/ETH 1% Pool

Fee Tier: 1%; Tick Spacing: 200; Token0: USD Coin; Token1: Wrapped Ether.

In [ ]:
sample_transport = RequestsHTTPTransport(
    url='https://api.thegraph.com/subgraphs/name/uniswap/uniswap-v3',
    verify=True,
    retries=5,
)
client = Client(
    transport=sample_transport
)
pool = "0x7bea39867e4169dbe237d55c8242a8f2fcdcc387"

In [28]:
timestamp_lt = 1659283200 # Mon Aug 01 2022 00:00:00 GMT+0800 (中国标准时间)
timestamp_gt = 1627747200 # Sun Aug 01 2021 00:00:00 GMT+0800 (中国标准时间)
mint_list_data = []

while timestamp_lt > timestamp_gt:
    for i in range(0,6):
        skip = i * 1000
        query_body = '''
        {
            mints(skip:%i,first:1000,orderBy: timestamp, orderDirection: desc, where:
             { pool:"0x7bea39867e4169dbe237d55c8242a8f2fcdcc387",timestamp_lt:%i}
            ){
            transaction{
            blockNumber
            timestamp
            gasUsed
            gasPrice
            }
            amount0
            amount1
            amountUSD
            tickLower
            tickUpper
            }
            }
        '''%(skip,timestamp_lt)
        query_request = gql(query_body)
        if skip == 0:
            mint_origin_data = client.execute(query_request)
            mint_origin_data = mint_origin_data['mints']
        else:
            mint_origin_data.extend(client.execute(query_request)['mints'])
    mint_list_data.extend(mint_origin_data)
    timestamp_lt = int(mint_list_data[-1]['transaction']['timestamp'])

while int(mint_list_data[-1]['transaction']['timestamp']) < timestamp_gt:
    del mint_list_data[-1]
print(mint_list_data[-1]['transaction']['timestamp'])
print(len(mint_list_data))

df_mint = pd.DataFrame(mint_list_data)
df_mint.drop(df_mint.columns[0],axis=1,inplace=True)
list_blockNumber = []
list_timestamp = []
list_gasUsed = []
list_gasPrice = []
for i in range(0,len(mint_list_data)):
    list_blockNumber.append(int(mint_list_data[i]['transaction']['blockNumber']))
    list_timestamp.append(int(mint_list_data[i]['transaction']['timestamp']))
    list_gasUsed.append(int(mint_list_data[i]['transaction']['gasUsed']))
    list_gasPrice.append(int(mint_list_data[i]['transaction']['gasPrice']))
df_mint.insert(0,'timestamp',list_timestamp)
df_mint.insert(1,'blockNumber',list_blockNumber)
df_mint.insert(2,'gasUsed',list_gasUsed)
df_mint.insert(3,'gasPrice',list_gasPrice)
for i in range(0,len(mint_list_data)):
    df_mint['timestamp'][i] = datetime.datetime.fromtimestamp(df_mint['timestamp'][i])
df_mint.rename(columns={'timestamp':'Date'},inplace=True)

df_mint.to_csv('tx_usdc_eth_10000_mints.csv', index = False, header = True)

1627772389
601


C:\Users\Administrator\AppData\Local\Temp\ipykernel_8424\517455345.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_mint['timestamp'][i] = datetime.datetime.fromtimestamp(df_mint['timestamp'][i])


In [29]:
timestamp_lt = 1659283200 # Mon Aug 01 2022 00:00:00 GMT+0800 (中国标准时间)
timestamp_gt = 1627747200 # Sun Aug 01 2021 00:00:00 GMT+0800 (中国标准时间)
burn_list_data = []

while timestamp_lt > timestamp_gt:
    for i in range(0,6):
        skip = i * 1000
        query_body = '''
        {
            burns(skip:%i,first:1000,orderBy: timestamp, orderDirection: desc, where:
             { pool:"0x7bea39867e4169dbe237d55c8242a8f2fcdcc387",timestamp_lt:%i}
            ){
            transaction{
            blockNumber
            timestamp
            gasUsed
            gasPrice
            }
            amount0
            amount1
            amountUSD
            tickLower
            tickUpper
            }
            }
        '''%(skip,timestamp_lt)
        query_request = gql(query_body)
        if skip == 0:
            burn_origin_data = client.execute(query_request)
            burn_origin_data = burn_origin_data['burns']
        else:
            burn_origin_data.extend(client.execute(query_request)['burns'])
    burn_list_data.extend(burn_origin_data)
    timestamp_lt = int(burn_list_data[-1]['transaction']['timestamp'])

while int(burn_list_data[-1]['transaction']['timestamp']) < timestamp_gt:
    del burn_list_data[-1]

print(burn_list_data[-1]['transaction']['timestamp'])
print(len(burn_list_data))

df_burn = pd.DataFrame(burn_list_data)
df_burn.drop(df_burn.columns[0],axis=1,inplace=True)
list_blockNumber = []
list_timestamp = []
list_gasUsed = []
list_gasPrice = []
for i in range(0,len(burn_list_data)):
    list_blockNumber.append(int(burn_list_data[i]['transaction']['blockNumber']))
    list_timestamp.append(int(burn_list_data[i]['transaction']['timestamp']))
    list_gasUsed.append(int(burn_list_data[i]['transaction']['gasUsed']))
    list_gasPrice.append(int(burn_list_data[i]['transaction']['gasPrice']))
df_burn.insert(0,'timestamp',list_timestamp)
df_burn.insert(1,'blockNumber',list_blockNumber)
df_burn.insert(2,'gasUsed',list_gasUsed)
df_burn.insert(3,'gasPrice',list_gasPrice)
for i in range(0,len(burn_list_data)):
    df_burn['timestamp'][i] = datetime.datetime.fromtimestamp(df_burn['timestamp'][i])
df_burn.rename(columns={'timestamp':'Date'},inplace=True)

df_burn.to_csv('tx_usdc_eth_10000_burns.csv', index = False, header = True)

1627772037
640


C:\Users\Administrator\AppData\Local\Temp\ipykernel_8424\1579148241.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_burn['timestamp'][i] = datetime.datetime.fromtimestamp(df_burn['timestamp'][i])


In [32]:
timestamp_lt = 1659283200 # Mon Aug 01 2022 00:00:00 GMT+0800 (中国标准时间)
timestamp_gt = 1627747200 # Sun Aug 01 2021 00:00:00 GMT+0800 (中国标准时间)
swap_list_data = []
while timestamp_lt > timestamp_gt:
    for i in range(0,6):
        skip = i * 1000
        query_body = '''
        {
            swaps(skip:%i,first:1000,orderBy: timestamp, orderDirection: desc, where:
             { pool:"0x7bea39867e4169dbe237d55c8242a8f2fcdcc387",timestamp_lt:%i}
            ){
            transaction{
            blockNumber
            timestamp
            gasUsed
            gasPrice
            }
            amount0
            amount1
            amountUSD
            sqrtPriceX96
            tick
            }
            }
        '''%(skip,timestamp_lt)
        query_request = gql(query_body)
        if skip == 0:
            swap_origin_data = client.execute(query_request)
            swap_origin_data = swap_origin_data['swaps']
        else:
            swap_origin_data.extend(client.execute(query_request)['swaps'])
    swap_list_data.extend(swap_origin_data)
    timestamp_lt = int(swap_list_data[-1]['transaction']['timestamp'])

while int(swap_list_data[-1]['transaction']['timestamp']) < timestamp_gt:
    del swap_list_data[-1]


print(swap_list_data[-1]['transaction']['timestamp'])
print(len(swap_list_data))

df_swap = pd.DataFrame(swap_list_data)
df_swap.drop(df_swap.columns[0],axis=1,inplace=True)
list_blockNumber = []
list_timestamp = []
list_gasUsed = []
list_gasPrice = []
for i in range(0,len(swap_list_data)):
    list_blockNumber.append(int(swap_list_data[i]['transaction']['blockNumber']))
    list_timestamp.append(int(swap_list_data[i]['transaction']['timestamp']))
    list_gasUsed.append(int(swap_list_data[i]['transaction']['gasUsed']))
    list_gasPrice.append(int(swap_list_data[i]['transaction']['gasPrice']))
df_swap.insert(0,'timestamp',list_timestamp)
df_swap.insert(1,'blockNumber',list_blockNumber)
df_swap.insert(2,'gasUsed',list_gasUsed)
df_swap.insert(3,'gasPrice',list_gasPrice)
for i in range(0,len(swap_list_data)):
    df_swap['timestamp'][i] = datetime.datetime.fromtimestamp(df_swap['timestamp'][i])
df_swap.rename(columns={'timestamp':'Date'},inplace=True)
df_swap.to_csv('tx_usdc_eth_10000_swaps.csv', index = False, header = True)

1627763836
18330


C:\Users\Administrator\AppData\Local\Temp\ipykernel_8424\4163767354.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_swap['timestamp'][i] = datetime.datetime.fromtimestamp(df_swap['timestamp'][i])


In [33]:
query_body = '''
{
    flashes(first:1000,orderBy: timestamp, orderDirection: desc, where:
     { pool: "0x7bea39867e4169dbe237d55c8242a8f2fcdcc387" }
    ){
    transaction{
    blockNumber
    timestamp
    gasUsed
    gasPrice
    }
    amount0
    amount1
    amountUSD
    amount0Paid
    amount1Paid
    }
    }
'''
query_request = gql(query_body)
flash_origin_data = client.execute(query_request)
print(flash_origin_data)

{'flashes': []}


In [34]:
query_body = '''
{
    collects(first:1000,orderBy: timestamp, orderDirection: desc, where:
     { pool: "0x7bea39867e4169dbe237d55c8242a8f2fcdcc387" }
    ){
    transaction{
    blockNumber
    timestamp
    gasUsed
    gasPrice
    }
    amount0
    amount1
    amountUSD
    tickLower
    tickUpper
    }
    }
'''
query_request = gql(query_body)
collect_origin_data = client.execute(query_request)
print(collect_origin_data)

{'collects': []}


In [35]:
df_mint = pd.read_csv('tx_usdc_eth_10000_mints.csv')
df_burn = pd.read_csv('tx_usdc_eth_10000_burns.csv')
df_swap = pd.read_csv('tx_usdc_eth_10000_swaps.csv')

In [36]:
type1 = []
type2 = []
type3 = []

for i in range(0,len(df_mint['Date'])):
    type1.append('Mint')
for i in range(0,len(df_burn['Date'])):
    type2.append('Burn')
for i in range(0,len(df_swap['Date'])):
    type3.append('Swap')
    
df_mint['type'] = type1
df_burn['type'] = type2
df_swap['type'] = type3

df = pd.merge(df_mint,df_burn,how='outer')
df = pd.merge(df,df_swap,how='outer')
df = df.sort_values(by=['Date']).reset_index()
df.drop(df.columns[0],axis=1,inplace=True)
df

,Date,blockNumber,gasUsed,gasPrice,amount0,amount1,amountUSD,tickLower,tickUpper,type,sqrtPriceX96,tick
0,2021-08-01 04:37:16,12935445,525540,48404510010,11675.906633,-4.704228,11677.009027,NaN,NaN,Swap,1596771632554108937635653191947007,198233.0
1,2021-08-01 04:37:58,12935450,1200000,1,4787.051886,-1.923479,4781.144709,NaN,NaN,Swap,1595513507448044608540680053000000,198217.0
2,2021-08-01 04:38:07,12935452,303511,38000000000,12280.464267,-4.920723,12248.336920,NaN,NaN,Swap,1592385978457148793797449073505500,198178.0
3,2021-08-01 05:07:02,12935575,301423,60000000000,2197.161132,-0.878387,2196.311527,NaN,NaN,Swap,1591840003399468840142334558969306,198171.0
4,2021-08-01 05:07:39,12935579,207340,55000000000,2567.058294,-1.025502,2567.333433,NaN,NaN,Swap,1591202586230063512229824820059106,198163.0
...,...,...,...,...,...,...,...,...,...,...,...,...
19566,2022-07-31 18:58:57,15249837,252196,30912107293,34595.100000,-20.161025,34588.952885,NaN,NaN,Swap,1921448417750268436516605481507231,201935.0
19567,2022-07-31 18:58:57,15249837,900000,11182878472,27833.502631,-16.196019,27807.507833,NaN,NaN,Swap,1920152242584205833483422364000000,201921.0
19568,2022-07-31 19:10:22,15249890,252196,51942504683,46820.700000,-27.195229,46780.032469,NaN,NaN,Swap,1917975795420808176096137830751193,201899.0
19569,2022-07-31 20:21:30,15250200,544009,8760640941,226.059078,0.205691,579.126205,200800.0,202600.0,Mint,NaN,NaN


In [37]:
df = df[['Date', 'blockNumber', 'gasUsed', 'gasPrice', 'amount0', 'amount1',
       'amountUSD', 'tickLower', 'tickUpper', 'sqrtPriceX96', 'tick', 'type']]
df

,Date,blockNumber,gasUsed,gasPrice,amount0,amount1,amountUSD,tickLower,tickUpper,sqrtPriceX96,tick,type
0,2021-08-01 04:37:16,12935445,525540,48404510010,11675.906633,-4.704228,11677.009027,NaN,NaN,1596771632554108937635653191947007,198233.0,Swap
1,2021-08-01 04:37:58,12935450,1200000,1,4787.051886,-1.923479,4781.144709,NaN,NaN,1595513507448044608540680053000000,198217.0,Swap
2,2021-08-01 04:38:07,12935452,303511,38000000000,12280.464267,-4.920723,12248.336920,NaN,NaN,1592385978457148793797449073505500,198178.0,Swap
3,2021-08-01 05:07:02,12935575,301423,60000000000,2197.161132,-0.878387,2196.311527,NaN,NaN,1591840003399468840142334558969306,198171.0,Swap
4,2021-08-01 05:07:39,12935579,207340,55000000000,2567.058294,-1.025502,2567.333433,NaN,NaN,1591202586230063512229824820059106,198163.0,Swap
...,...,...,...,...,...,...,...,...,...,...,...,...
19566,2022-07-31 18:58:57,15249837,252196,30912107293,34595.100000,-20.161025,34588.952885,NaN,NaN,1921448417750268436516605481507231,201935.0,Swap
19567,2022-07-31 18:58:57,15249837,900000,11182878472,27833.502631,-16.196019,27807.507833,NaN,NaN,1920152242584205833483422364000000,201921.0,Swap
19568,2022-07-31 19:10:22,15249890,252196,51942504683,46820.700000,-27.195229,46780.032469,NaN,NaN,1917975795420808176096137830751193,201899.0,Swap
19569,2022-07-31 20:21:30,15250200,544009,8760640941,226.059078,0.205691,579.126205,200800.0,202600.0,NaN,NaN,Mint


In [38]:
df.to_csv('tx_usdc_eth_10000.csv', index = False, header = True)

# Some Calculations

In [9]:
import pandas as pd

In [3]:
df1 = pd.read_csv('tx_usdc_eth_500.csv')
df2 = pd.read_csv('tx_usdc_eth_3000.csv')
df3 = pd.read_csv('tx_usdc_eth_10000.csv')

In [ ]:
df1

In [5]:
df2

,Date,blockNumber,gasUsed,gasPrice,amount0,amount1,amountUSD,tickLower,tickUpper,sqrtPriceX96,tick,type
0,2021-08-01 00:13:00,12934313,1200000,1694000000001,1.129678e+06,-458.879595,1.127205e+06,NaN,NaN,1598099139479430347978933773504862,198249.0,Swap
1,2021-08-01 00:31:14,12934406,520046,75000000000,2.000000e+06,158.709806,2.390082e+06,198240.0,198300.0,NaN,NaN,Mint
2,2021-08-01 00:43:04,12934458,800000,0,1.294616e+07,208.234461,1.345835e+07,198240.0,198300.0,NaN,NaN,Burn
3,2021-08-01 00:43:04,12934458,800000,0,1.126098e+07,893.614359,1.345734e+07,198240.0,198300.0,NaN,NaN,Mint
4,2021-08-01 00:43:04,12934458,600000,70000000000,-1.497920e+04,6.108293,1.500181e+04,NaN,NaN,1597514522549451398125129554190367,198242.0,Swap
...,...,...,...,...,...,...,...,...,...,...,...,...
263741,2022-07-31 23:41:35,15251115,312243,8416887908,8.053142e+02,-0.470167,8.041056e+02,NaN,NaN,1917232987762010505017082710333290,201891.0,Swap
263742,2022-07-31 23:42:23,15251117,607963,10884777864,5.000000e+02,-0.291914,4.992497e+02,NaN,NaN,1917231021200302815741896480321151,201891.0,Swap
263743,2022-07-31 23:44:23,15251125,374484,13581742365,2.928553e+03,2.843247,7.783952e+03,197940.0,204180.0,NaN,NaN,Burn
263744,2022-07-31 23:49:28,15251152,1089650,10698263948,2.173966e+05,-126.865645,2.170222e+05,NaN,NaN,1916376307864556945633878900100759,201882.0,Swap


In [7]:
df2['price'] = 1/(1.0001**df2['tick'])*10**12
df2

,Date,blockNumber,gasUsed,gasPrice,amount0,amount1,amountUSD,tickLower,tickUpper,sqrtPriceX96,tick,type,price
0,2021-08-01 00:13:00,12934313,1200000,1694000000001,1.129678e+06,-458.879595,1.127205e+06,NaN,NaN,1598099139479430347978933773504862,198249.0,Swap,2458.022134
1,2021-08-01 00:31:14,12934406,520046,75000000000,2.000000e+06,158.709806,2.390082e+06,198240.0,198300.0,NaN,NaN,Mint,NaN
2,2021-08-01 00:43:04,12934458,800000,0,1.294616e+07,208.234461,1.345835e+07,198240.0,198300.0,NaN,NaN,Burn,NaN
3,2021-08-01 00:43:04,12934458,800000,0,1.126098e+07,893.614359,1.345734e+07,198240.0,198300.0,NaN,NaN,Mint,NaN
4,2021-08-01 00:43:04,12934458,600000,70000000000,-1.497920e+04,6.108293,1.500181e+04,NaN,NaN,1597514522549451398125129554190367,198242.0,Swap,2459.743265
...,...,...,...,...,...,...,...,...,...,...,...,...,...
263741,2022-07-31 23:41:35,15251115,312243,8416887908,8.053142e+02,-0.470167,8.041056e+02,NaN,NaN,1917232987762010505017082710333290,201891.0,Swap,1707.747456
263742,2022-07-31 23:42:23,15251117,607963,10884777864,5.000000e+02,-0.291914,4.992497e+02,NaN,NaN,1917231021200302815741896480321151,201891.0,Swap,1707.747456
263743,2022-07-31 23:44:23,15251125,374484,13581742365,2.928553e+03,2.843247,7.783952e+03,197940.0,204180.0,NaN,NaN,Burn,NaN
263744,2022-07-31 23:49:28,15251152,1089650,10698263948,2.173966e+05,-126.865645,2.170222e+05,NaN,NaN,1916376307864556945633878900100759,201882.0,Swap,1709.285043


In [8]:
df2['tx_cost'] = df2['gasUsed']*df2['gasPrice']
df2

,Date,blockNumber,gasUsed,gasPrice,amount0,amount1,amountUSD,tickLower,tickUpper,sqrtPriceX96,tick,type,price,tx_cost
0,2021-08-01 00:13:00,12934313,1200000,1694000000001,1.129678e+06,-458.879595,1.127205e+06,NaN,NaN,1598099139479430347978933773504862,198249.0,Swap,2458.022134,2032800000001200000
1,2021-08-01 00:31:14,12934406,520046,75000000000,2.000000e+06,158.709806,2.390082e+06,198240.0,198300.0,NaN,NaN,Mint,NaN,39003450000000000
2,2021-08-01 00:43:04,12934458,800000,0,1.294616e+07,208.234461,1.345835e+07,198240.0,198300.0,NaN,NaN,Burn,NaN,0
3,2021-08-01 00:43:04,12934458,800000,0,1.126098e+07,893.614359,1.345734e+07,198240.0,198300.0,NaN,NaN,Mint,NaN,0
4,2021-08-01 00:43:04,12934458,600000,70000000000,-1.497920e+04,6.108293,1.500181e+04,NaN,NaN,1597514522549451398125129554190367,198242.0,Swap,2459.743265,42000000000000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
263741,2022-07-31 23:41:35,15251115,312243,8416887908,8.053142e+02,-0.470167,8.041056e+02,NaN,NaN,1917232987762010505017082710333290,201891.0,Swap,1707.747456,2628114331057644
263742,2022-07-31 23:42:23,15251117,607963,10884777864,5.000000e+02,-0.291914,4.992497e+02,NaN,NaN,1917231021200302815741896480321151,201891.0,Swap,1707.747456,6617542204531032
263743,2022-07-31 23:44:23,15251125,374484,13581742365,2.928553e+03,2.843247,7.783952e+03,197940.0,204180.0,NaN,NaN,Burn,NaN,5086145207814660
263744,2022-07-31 23:49:28,15251152,1089650,10698263948,2.173966e+05,-126.865645,2.170222e+05,NaN,NaN,1916376307864556945633878900100759,201882.0,Swap,1709.285043,11657363310938200


In [ ]:
df3